# 무역 데이터 EDA (HS4 / item_id 단위 EDA 및 micro-level deep dive)

**코드 셀 동작 설명**

- **주요 범주**: 라이브러리 임포트, 데이터 로드, 시각화(Seaborn), 수치/데이터프레임 처리
- 데이터 파일을 읽어 DataFrame으로 로드
- 그래프/차트로 결과 시각화

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


df_train = pd.read_csv('train.csv')
print(df_train.head(20))



     item_id  year  month  seq  type   hs4      weight  quantity       value
0   DEWLVASR  2022      1  1.0     1  3038     14858.0       0.0     32688.0
1   ELQGMQWE  2022      1  1.0     1  2002     62195.0       0.0    110617.0
2   AHMDUILJ  2022      1  1.0     1  2102     18426.0       0.0     72766.0
3   XIPPENFQ  2022      1  1.0     1  2501     20426.0       0.0     11172.0
4   FTSVTTSR  2022      1  1.0     1  2529    248000.0       0.0    143004.0
5   XMKRPGLB  2022      1  1.0     1  2529    558000.0       0.0    205676.0
6   RCBZUSIM  2022      1  1.0     1  2805         0.0       0.0       459.0
7   SUOYXCHP  2022      1  1.0     1  2805         0.0       0.0       482.0
8   ZKENOUDA  2022      1  1.0     1  2805         2.0       0.0      2783.0
9   WQMVCOEM  2022      1  1.0     1  2805       187.0       0.0    217777.0
10  BSRMSVTC  2022      1  1.0     1  2805       111.0       0.0     33952.0
11  DDEXPPXU  2022      1  1.0     1  2807       387.0       0.0     13158.0

## 1. 전처리

- zero / missing flag 생성:
  - value, weight, quantity의 0은 정상적일 수 있음 👉 제거 X
  - 대신 zero 패턴 자체를 feature로 인식하기 위해 flag 생성.

- 결측치 처리:
  - weight, quantity, value 결측은 보간하지 않음 (의미 있는 결측이기 때문)

- item_id / HS4 단위 monthly pivot 생성
  - → Comovement 분석의 기본 단위로 사용될 시계열 구조 확보.

**코드 셀 동작 설명**

- **주요 범주**: 결측치 확인/처리, 타입 변환/날짜 처리
- 결측치 상태를 확인하고 적절히 제거/대체
- 컬럼 타입 변환 또는 날짜 파싱 수행

In [2]:

# 삭제할 이상치 조건 생성
mask_outlier = (
    (df_train["item_id"] == "QVLMOEYE") &
    (df_train["year"] == 2023) &
    (df_train["month"] == 8) &
    (df_train["hs4"] == 4202) &
    (df_train["weight"] == 3197) &
    (df_train["quantity"] == 792598955) &
    (df_train["value"] == 660801)
)

print("삭제될 행 수:", mask_outlier.sum())

# 이상치 삭제
df_train = df_train[~mask_outlier].reset_index(drop=True)


삭제될 행 수: 1


In [3]:
mask_fix = (
    (df_train["item_id"] == "WHPUAOID") &
    (df_train["year"] == 2025) &
    (df_train["month"] == 1) &
    (df_train["hs4"] == 5512) &
    (df_train["weight"] == 1) &
    (df_train["quantity"] == 1) &
    (df_train["value"] == 0)
)

df_train.loc[mask_fix, ["weight", "quantity", "value"]] = 0


In [4]:
# Zero flag 생성
df_train['value_zero'] = (df_train['value'] == 0).astype(int)
df_train['weight_zero'] = (df_train['weight'] == 0).astype(int)
df_train['quantity_zero'] = (df_train['quantity'] == 0).astype(int)

# Missing flag 생성
df_train['value_missing'] = df_train['value'].isna().astype(int)
df_train['weight_missing'] = df_train['weight'].isna().astype(int)
df_train['quantity_missing'] = df_train['quantity'].isna().astype(int)

**코드 셀 동작 설명**

- **주요 범주**: 일반 전처리/로직 수행
- 주요 전처리/계산/출력 수행

In [5]:
hs2_map = {
    "01": "산동물",
    "02": "육과 식용설육",
    "03": "어패류",
    "04": "낙농품·조란·천연동물성 식품",
    "05": "기타 동물성 생산품",
    "06": "산수목·꽃",
    "07": "채소",
    "08": "과실·견과류",
    "09": "커피·향신료",

    "10": "곡물",
    "11": "밀가루·전분",
    "12": "채유용 종자·인삼",
    "13": "식물성 엑스",
    "14": "기타 식물성 생산품",
    "15": "동식물성 유지",
    "16": "육·어류 조제품",
    "17": "당류·설탕과자",
    "18": "코코아·초콜릿",
    "19": "곡물·곡분 주제품·빵류",

    "20": "채소·과실의 조제품",
    "21": "기타 조제식료품",
    "22": "음료·주류·식초",
    "23": "조제사료",
    "24": "담배",
    "25": "토석류·소금",
    "26": "광·슬랙·회",
    "27": "광물성 연료·에너지",
    "28": "무기화합물",
    "29": "유기화합물",

    "30": "의료용품",
    "31": "비료",
    "32": "염료·안료·페인트·잉크",
    "33": "향료·화장품",
    "34": "비누·계면활성제·왁스",
    "35": "카세인·알부민·변성전분·효소",
    "36": "화약류·성냥",
    "37": "필름·인화지·사진용 재료",
    "38": "기타 화학공업 품목",
    "39": "플라스틱과 그 제품",

    "40": "고무와 그 제품",
    "41": "원피·가죽",
    "42": "가죽제품",
    "43": "모피·모피제품",
    "44": "목재·목탄",
    "45": "코르크·코르크 제품",
    "46": "조물재료 제품",
    "47": "펄프",
    "48": "지·판지",
    "49": "서적·신문·인쇄물",

    "50": "견·견사·견직물",
    "51": "양모·수모",
    "52": "면·면사·면직물",
    "53": "마류사·직물",
    "54": "인조 필라멘트 섬유",
    "55": "인조 스테이플 섬유",
    "56": "워딩·부직포",
    "57": "양탄자",
    "58": "특수 직물",
    "59": "침투·도포 직물",

    "60": "편물",
    "61": "의류(편물제)",
    "62": "의류(편물제 외)",
    "63": "기타 섬유제품·넝마",
    "64": "신발류",
    "65": "모자류",
    "66": "우산·지팡이",
    "67": "조제 우모·인조제품",
    "68": "석·시멘트·석면 제품",
    "69": "도자 제품",

    "70": "유리",
    "71": "귀석·반귀석·귀금속",
    "72": "철강",
    "73": "철강제품",
    "74": "동과 그 제품",
    "75": "니켈과 그 제품",
    "76": "알루미늄과 그 제품",
    "78": "연과 그 제품",
    "79": "아연과 그 제품",

    "80": "주석과 그 제품",
    "81": "기타 비금속",
    "82": "비금속 공구·스푼·포크",
    "83": "각종 비금속 제품",
    "84": "보일러·기계류",
    "85": "전기기기·TV·VTR",
    "86": "철도차량",
    "87": "일반차량",
    "88": "항공기",
    "89": "선박",

    "90": "광학·의료·측정·정밀기기",
    "91": "시계",
    "92": "악기",
    "93": "무기",
    "94": "가구·조명기구",
    "95": "완구·운동용품",
    "96": "잡품",
    "97": "예술품·골동품"
}


### 모든 item_id × 모든 month 패널 완성(Panel Completion)

현재 데이터는 모든 item_id가 모든 월에 등장하지 않음 → 공행성 분석에서 큰 문제됨.
동시에 존재하는 시계열끼리 비교해야 함

예:
- A item은 2022~2025 전 기간 존재
- B item은 특정 계절만 존재
- C item은 중간에 구멍 있음

이런 상태에서 comovement correlation 계산하면 시계열 길이가 불일치 → 오류 / 왜곡 발생, 상관계수 계산 불가<br>
- A : 2022.01 ~ 2025.07 모든 달에 존재 = 42개월
- C : 2022.03 ~ 2023.05 일부 달에 존재 = 15개월
→ 반드시 해야 하는 전처리:<br>
모든 item_id × 모든 month 조합을 만들고 없는 row는 0 또는 NaN으로 채우기

✅ 1단계: df_train에 date 컬럼 붙이기

아래 셀을 통째로 새로 실행해 줘:

✅ 2단계: panel(모든 item_id × 모든 month) 만들기

이제 date가 생겼다고 가정하고, 아래 셀 실행 👇

**코드 셀 동작 설명**

- **주요 범주**: 라이브러리 임포트, 결측치 확인/처리, 중복 데이터 처리, 타입 변환/날짜 처리, 집계/그룹 연산, 데이터 결합/병합, 정렬, 수치/데이터프레임 처리
- 결측치 상태를 확인하고 적절히 제거/대체
- 중복 행을 탐지하여 제거
- 컬럼 타입 변환 또는 날짜 파싱 수행
- 그룹 기준으로 집계/통계 계산
- 키를 기준으로 데이터 병합 또는 이어붙이기

In [6]:
import pandas as pd

# ------------------------------------
# 0) date 컬럼 만들기 (year, month 기준)
# ------------------------------------
# year, month가 있고 date가 아직 없을 때만 생성
if "date" not in df_train.columns:
    df_train["date"] = pd.to_datetime(
        df_train["year"].astype(int).astype(str) + "-" +
        df_train["month"].astype(int).astype(str) + "-01"
    )

# 혹시 date가 object/string인 경우 datetime으로 변환
df_train["date"] = pd.to_datetime(df_train["date"])

# ------------------------------------
# 1) 월별 집계 (중복거래 제거 핵심)
#    item_id + hs4 + date 기준으로 value/weight/quantity 합산
# ------------------------------------
agg_cols = ["value", "weight", "quantity"]

df_train_monthly = (
    df_train
    .groupby(["item_id", "hs4", "date"], as_index=False)[agg_cols]
    .sum()
)

print("월별 집계 결과 예시:")
print(df_train_monthly.head())

# ------------------------------------
# 2) 전체 month 범위 & item_id 목록 만들기
# ------------------------------------
min_month = df_train_monthly["date"].min()
max_month = df_train_monthly["date"].max()

full_months = pd.date_range(min_month, max_month, freq="MS")
all_items = df_train_monthly["item_id"].unique()

# ------------------------------------
# 3) item_id × date 전체 조합 (풀 패널 인덱스)
# ------------------------------------
panel = pd.MultiIndex.from_product(
    [all_items, full_months],
    names=["item_id", "date"]
).to_frame(index=False)

# ------------------------------------
# 4) 집계된 df_train_monthly와 merge해서 패널 완성
# ------------------------------------
df_panel = (
    panel.merge(df_train_monthly, on=["item_id", "date"], how="left")
         .sort_values(["item_id", "date"])
         .reset_index(drop=True)
)

# 거래 없던 month → NaN을 0으로 채우기 (원하면)
for c in agg_cols:
    df_panel[c] = df_panel[c].fillna(0)

# 연/월 컬럼 다시 생성
df_panel["year"] = df_panel["date"].dt.year
df_panel["month"] = df_panel["date"].dt.month

print("최종 패널 예시:")
print(df_panel.head())

# 중복 체크 (item_id, date 조합이 유일한지)
print("중복 row 개수:", df_panel.duplicated(["item_id", "date"]).sum())


월별 집계 결과 예시:
    item_id   hs4       date    value    weight  quantity
0  AANGBULD  4810 2022-01-01  14276.0   17625.0       0.0
1  AANGBULD  4810 2022-02-01  52347.0   67983.0       0.0
2  AANGBULD  4810 2022-03-01  53549.0   69544.0       0.0
3  AANGBULD  4810 2022-05-01  26997.0   34173.0       0.0
4  AANGBULD  4810 2022-06-01  84489.0  103666.0       0.0
최종 패널 예시:
    item_id       date     hs4    value   weight  quantity  year  month
0  AANGBULD 2022-01-01  4810.0  14276.0  17625.0       0.0  2022      1
1  AANGBULD 2022-02-01  4810.0  52347.0  67983.0       0.0  2022      2
2  AANGBULD 2022-03-01  4810.0  53549.0  69544.0       0.0  2022      3
3  AANGBULD 2022-04-01     NaN      0.0      0.0       0.0  2022      4
4  AANGBULD 2022-05-01  4810.0  26997.0  34173.0       0.0  2022      5
중복 row 개수: 0


**코드 셀 동작 설명**

- **주요 범주**: 중복 데이터 처리
- 중복 행을 탐지하여 제거

In [7]:
df_panel.duplicated(["item_id", "date"]).sum()

np.int64(0)

**코드 셀 동작 설명**

- **주요 범주**: 타입 변환/날짜 처리
- 컬럼 타입 변환 또는 날짜 파싱 수행

In [8]:
df_panel["hs2"] = df_panel["hs4"].astype(str).str[:2]
df_panel["hs2_name_kr"] = df_panel["hs2"].map(hs2_map)

**코드 셀 동작 설명**

- **주요 범주**: 결측치 확인/처리, 중복 데이터 처리
- 결측치 상태를 확인하고 적절히 제거/대체
- 중복 행을 탐지하여 제거

In [9]:
#df_panel.isnull().sum()
#df_panel[df_panel['seq'].isnull()]

#df_panel.duplicated().sum()
df_panel.head()


,item_id,date,hs4,value,weight,quantity,year,month,hs2,hs2_name_kr
0,AANGBULD,2022-01-01,4810.0,14276.0,17625.0,0.0,2022,1,48,지·판지
1,AANGBULD,2022-02-01,4810.0,52347.0,67983.0,0.0,2022,2,48,지·판지
2,AANGBULD,2022-03-01,4810.0,53549.0,69544.0,0.0,2022,3,48,지·판지
3,AANGBULD,2022-04-01,NaN,0.0,0.0,0.0,2022,4,na,NaN
4,AANGBULD,2022-05-01,4810.0,26997.0,34173.0,0.0,2022,5,48,지·판지


**코드 셀 동작 설명**

- **주요 범주**: 결측치 확인/처리, 타입 변환/날짜 처리
- 결측치 상태를 확인하고 적절히 제거/대체
- 컬럼 타입 변환 또는 날짜 파싱 수행

In [10]:
# 거래량 관련 수치 컬럼
value_cols = ['value', 'weight', 'quantity']

# 없는 월은 거래 0으로 (너 프로젝트 설정에 따라 조정 가능)
df_panel[value_cols] = df_panel[value_cols].fillna(0)

# zero / missing flag가 이미 df_train에 있다면 같이 채워주기
flag_cols = [
    'value_zero', 'weight_zero', 'quantity_zero',
    'value_missing', 'weight_missing', 'quantity_missing'
]

for c in flag_cols:
    if c in df_panel.columns:
        df_panel[c] = df_panel[c].fillna(0).astype(int)

df_panel.head()
df_panel[df_panel['item_id'] == 'GKQIJYDH']

,item_id,date,hs4,value,weight,quantity,year,month,hs2,hs2_name_kr
1247,GKQIJYDH,2022-01-01,8501.0,9782469.0,579404.0,68.0,2022,1,85,전기기기·TV·VTR
1248,GKQIJYDH,2022-02-01,8501.0,321176.0,50685.0,31.0,2022,2,85,전기기기·TV·VTR
1249,GKQIJYDH,2022-03-01,8501.0,7106218.0,416214.0,68.0,2022,3,85,전기기기·TV·VTR
1250,GKQIJYDH,2022-04-01,8501.0,17850488.0,854306.0,550.0,2022,4,85,전기기기·TV·VTR
1251,GKQIJYDH,2022-05-01,8501.0,41768869.0,3025992.0,181.0,2022,5,85,전기기기·TV·VTR
1252,GKQIJYDH,2022-06-01,8501.0,11677001.0,698063.0,125.0,2022,6,85,전기기기·TV·VTR
1253,GKQIJYDH,2022-07-01,8501.0,8420099.0,630011.0,88.0,2022,7,85,전기기기·TV·VTR
1254,GKQIJYDH,2022-08-01,8501.0,14228993.0,794488.0,108.0,2022,8,85,전기기기·TV·VTR
1255,GKQIJYDH,2022-09-01,8501.0,19845420.0,741487.0,110.0,2022,9,85,전기기기·TV·VTR
1256,GKQIJYDH,2022-10-01,8501.0,9518385.0,378466.0,123.0,2022,10,85,전기기기·TV·VTR


## 2. HS4 / item_id 군집성 분석

**코드 셀 동작 설명**

- **주요 범주**: 집계/그룹 연산, 정렬
- 그룹 기준으로 집계/통계 계산

In [11]:
df = df_panel.copy() 

hs4_summary = (
    df.groupby("hs4")[["value", "weight"]]
      .mean()
      .reset_index()
      .sort_values("value", ascending=False)
)

hs4_summary.head(10)
# 어떤 HS4 품목군의 평균 거래금액과 평균 중량이 높은지 파악
# 품목군의 경제적 규모 이해

,hs4,value,weight
11,2814.0,5.999737e+07,1.022512e+08
54,6211.0,2.516274e+07,8.551477e+05
32,3824.0,2.459375e+07,8.139476e+06
23,3102.0,1.754666e+07,3.233097e+07
70,9403.0,1.721897e+07,6.029486e+06
65,8505.0,1.216691e+07,3.952844e+05
31,3815.0,1.050937e+07,9.385711e+05
63,8479.0,6.980014e+06,3.525974e+05
14,2836.0,6.607692e+06,2.160214e+07
64,8501.0,6.485218e+06,4.014716e+05


**코드 셀 동작 설명**

- **주요 범주**: 집계/그룹 연산, 컬럼 이름 변경/추가, 정렬
- 그룹 기준으로 집계/통계 계산

In [12]:

hs4_qty_rate = (
    df.assign(qty_valid=(df["quantity"] > 0))
      .groupby("hs4")["qty_valid"]
      .mean()
      .reset_index(name="quantity_usage_rate")
      .sort_values("quantity_usage_rate", ascending=False)
)

hs4_qty_rate.head(10)
# 어떤 HS4는 quantity가 자주 기록되지만
# 어떤 HS4는 quantity가 거의 0 또는 누락
# 품목군 특성 파악, 모델링 시 quantity를 feature로 사용할지 판단

,hs4,quantity_usage_rate
66,8527.0,1.0
62,8467.0,1.0
69,9022.0,1.0
37,4403.0,1.0
35,4202.0,1.0
36,4302.0,1.0
7,2710.0,1.0
61,8461.0,1.0
47,5512.0,1.0
53,6101.0,1.0


**코드 셀 동작 설명**

- **주요 범주**: 집계/그룹 연산, 정렬
- 그룹 기준으로 집계/통계 계산

In [13]:
item_volume = (
    df.groupby("item_id")["value"]
      .sum()
      .reset_index()
      .sort_values("value", ascending=False)
)

top_items = item_volume.head(20)
bottom_items = item_volume.tail(20)

top_items, bottom_items

# 가장 거래규모가 큰 품목과 작은 품목
# 예측 우선순위 선정, 군집분석 시 중요


(     item_id         value
 21  FCYBOAXC  4.671997e+09
 4   ATLDMDBO  2.579887e+09
 31  GYHKIVQT  1.456760e+09
 12  CCLHWFWF  1.081998e+09
 74  STZDBITS  1.057606e+09
 45  LRVGFDFM  8.040184e+08
 11  BUZIIBYG  7.404158e+08
 10  BTMOEMEP  7.049946e+08
 29  GKQIJYDH  6.070892e+08
 72  SDWAYPIK  4.381489e+08
 88  WPQXWHYO  4.093971e+08
 42  KJNSOAHR  3.001406e+08
 17  DNMPSKTB  2.888946e+08
 33  HXYSSRXE  2.841308e+08
 48  LUENUFGA  2.756382e+08
 37  JPBRUTWP  2.664403e+08
 61  QRKRBYJL  2.214571e+08
 6   BEZYMBBT  2.123930e+08
 20  EVBVXETX  1.975996e+08
 94  XUOIQPFL  1.960189e+08,
      item_id     value
 14  DDEXPPXU  820507.0
 38  JSLXRQOK  651539.0
 34  IGDVVKUD  606705.0
 8   BLANHGYY  541289.0
 65  RCBZUSIM  540481.0
 58  PYZMVUWD  530103.0
 99  ZXERAXWP  485789.0
 2   ANWUJOKX  350309.0
 36  JERHKLYW  334445.0
 22  FDXPMYGF  219744.0
 51  NAQIHUKZ  202508.0
 27  FWUCPMMW   97723.0
 95  YSYHGLQK   95838.0
 28  GIKPEWTY   94461.0
 16  DJBLNPNC   85293.0
 57  PLMZALFA    5385.0
 76

**코드 셀 동작 설명**

- **주요 범주**: 집계/그룹 연산, 정렬
- 그룹 기준으로 집계/통계 계산

In [14]:
item_month_coverage = (
    df.groupby("item_id")["date"]
      .nunique()
      .reset_index(name="month_covered")
      .sort_values("month_covered", ascending=False)
)

item_month_coverage.head()
# 모든 item_id가 모든 월에 등장하는지 확인
# 모든 월에 등장하는 item_id만 모델링에 활용

#item_month_coverage.tail()
# 모든 월에 등장하지 않는 item_id는 모델링에서 제외


,item_id,month_covered
0,AANGBULD,43
1,AHMDUILJ,43
2,ANWUJOKX,43
3,APQGTRMF,43
4,ATLDMDBO,43


In [15]:
# -------------------------------
# 3) Pair-level RandomForest 모델 학습
# -------------------------------

def build_pair_features_for_training(
    feat_df: pd.DataFrame,
    candidates_df: pd.DataFrame,
    target_month: pd.Timestamp,
    feature_cols: list[str],
) -> tuple[pd.DataFrame, pd.Series]:
    """
    target_month = t+1 (예: 2025-08-01)
    t 시점 feature로 t+1의 following_item value를 예측하기 위한 X, y를 생성.
    """
    df = feat_df.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    base = df.set_index(["item_id", "date"])

    # Series나 dict 모두 처리하는 헬퍼 함수
    def safe_get(data, key, default=np.nan):
        if hasattr(data, 'get'):
            return data.get(key, default)
        elif hasattr(data, '__getitem__'):
            try:
                return data[key]
            except (KeyError, IndexError):
                return default
        else:
            return default

    rows = []
    y_values = []

    for _, pair in candidates_df.iterrows():
        try:
            lead = pair["leading_item_id"]
            foll = pair["following_item_id"]
            t = target_month - pd.DateOffset(months=1)  # t = target_month - 1

            # None이나 NaN 체크
            if pd.isna(lead) or pd.isna(foll):
                continue

            if (foll, t) not in base.index or (lead, t) not in base.index:
                continue
            if (foll, target_month) not in base.index:
                continue

            foll_t = base.loc[(foll, t)]
            lead_t = base.loc[(lead, t)]
            foll_t1 = base.loc[(foll, target_month)]  # t+1의 following value

            row = {
                "leading_item_id": lead,
                "following_item_id": foll,
                "date_t": t,
                "year": safe_get(foll_t, "year"),
                "month": safe_get(foll_t, "month"),
                "trend_index": safe_get(foll_t, "trend_index"),
            }

            # feature_cols에 포함된 피처들을 pair 형태로 변환
            for col in feature_cols:
                try:
                    if col.startswith("f_"):
                        base_name = col[2:]
                        row[col] = safe_get(foll_t, base_name)
                    elif col.startswith("l_"):
                        base_name = col[2:]
                        row[col] = safe_get(lead_t, base_name)
                    elif col in ["year", "month", "trend_index", "month_sin", "month_cos"]:
                        # 공유 피처는 following 기준으로 사용
                        if col not in row:
                            row[col] = safe_get(foll_t, col)
                except Exception:
                    row[col] = np.nan

            rows.append(row)
            # log_value를 타겟으로 사용 (predict_aug_2025_values_v2와 일치)
            log_value_t1 = safe_get(foll_t1, "log_value", np.nan)
            if pd.isna(log_value_t1):
                # log_value가 없으면 value에서 계산
                value_t1 = safe_get(foll_t1, "value", 0)
                log_value_t1 = np.log1p(max(0, float(value_t1)))
            y_values.append(log_value_t1)
        except Exception:
            continue

    feat_pair_df = pd.DataFrame(rows)
    X = feat_pair_df[feature_cols].copy()
    y = pd.Series(y_values, index=feat_pair_df.index)

    return X, y


def train_pair_model_v2(
    feat_df: pd.DataFrame,
    candidates_df: pd.DataFrame,
    train_last_t: pd.Timestamp,
    valid_last_t: pd.Timestamp,
    n_estimators: int = 100,
    max_depth: int = 10,
    random_state: int = 42,
) -> tuple[RandomForestRegressor, list[str]]:
    """
    Pair-level RandomForest 모델 학습.
    
    - feat_df: add_ts_core_features까지 적용된 피처 패널
    - candidates_df: 공행성 후보쌍
    - train_last_t: 학습에 사용할 마지막 date
    - valid_last_t: 검증용 마지막 date (현재는 미사용)
    
    반환:
      - model: 학습된 RandomForestRegressor
      - feature_cols: 사용된 피처 컬럼 리스트
    """
    df = feat_df.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # 학습 기간의 모든 월 추출
    train_dates = sorted(df[df["date"] <= train_last_t]["date"].unique())
    
    if len(train_dates) < 2:
        raise ValueError("학습에 사용할 충분한 날짜가 없습니다.")

    # 피처 컬럼 정의: following과 leading의 피처를 구분
    # 확장된 피처 리스트 (더 많은 정보 활용)
    base_features = [
        "log_value", "log_weight", "log_quantity",
        "log_value_lag1", "log_value_lag2", "log_value_lag3", "log_value_lag6", "log_value_lag12",
        "log_weight_lag1", "log_weight_lag2", "log_weight_lag3",
        "log_roll_mean_3", "log_roll_mean_6", "log_roll_mean_12",
        "log_roll_std_3", "log_roll_std_6", "log_roll_std_12",
        "log_roll_min_3", "log_roll_max_3",
        "log_value_yoy", "log_value_yoy_pct",
        "value_ema_3", "value_ema_6", "value_ema_12",
        "hs4_value_mean_3m", "hs4_value_mean_6m", "hs4_value_mean_12m",
        "month_sin", "month_cos",
        "year", "month", "trend_index"
    ]
    
    # 실제 존재하는 피처만 사용
    available_features = [f for f in base_features if f in df.columns]
    
    # Pair 피처: f_ (following), l_ (leading)
    # 단일 피처 (year, month, trend_index, month_sin, month_cos)는 공유
    feature_cols = []
    for feat in available_features:
        if feat not in ["year", "month", "trend_index", "month_sin", "month_cos"]:
            feature_cols.append(f"f_{feat}")
            feature_cols.append(f"l_{feat}")
        else:
            feature_cols.append(feat)

    # 학습 데이터 생성: 각 월별로 pair feature와 다음 달 value를 수집
    X_train_list = []
    y_train_list = []
    failed_dates = []

    for i in range(len(train_dates) - 1):
        t = train_dates[i]
        t_next = train_dates[i + 1]
        
        try:
            X_t, y_t = build_pair_features_for_training(
                df, candidates_df, t_next, feature_cols
            )
            if len(X_t) > 0:
                X_train_list.append(X_t)
                y_train_list.append(y_t)
            else:
                failed_dates.append((t, t_next, "빈 데이터"))
        except Exception as e:
            failed_dates.append((t, t_next, str(e)))
            continue

    if len(X_train_list) == 0:
        error_msg = "학습 데이터를 생성할 수 없습니다.\n"
        error_msg += f"   - 시도한 날짜 쌍 수: {len(train_dates) - 1}\n"
        error_msg += f"   - 실패한 날짜 쌍 수: {len(failed_dates)}\n"
        if len(failed_dates) > 0:
            error_msg += f"   - 처음 5개 실패 사례:\n"
            for t, t_next, reason in failed_dates[:5]:
                error_msg += f"     * {t} -> {t_next}: {reason}\n"
        error_msg += "\n해결 방법:\n"
        error_msg += "   1. candidates_df에 유효한 쌍이 있는지 확인\n"
        error_msg += "   2. feat_df에 해당 날짜의 데이터가 있는지 확인\n"
        error_msg += "   3. train_last_t를 더 이른 날짜로 조정\n"
        raise ValueError(error_msg)
    
    if len(failed_dates) > 0:
        print(f"⚠️ {len(failed_dates)}개 날짜 쌍에서 데이터 생성 실패 (무시됨)")

    X_train = pd.concat(X_train_list, axis=0).reset_index(drop=True)
    y_train = pd.concat(y_train_list, axis=0).reset_index(drop=True)

    # NaN 처리 - y_train의 NaN은 제거 (log_value가 없으면 학습 불가)
    # X_train의 NaN은 0으로 채우되, y_train이 유효한 샘플만 사용
    valid_mask = ~(y_train.isna() | (y_train <= 0))
    
    if valid_mask.sum() == 0:
        raise ValueError("유효한 학습 샘플이 없습니다! (모든 y_train이 NaN이거나 0 이하)")
    
    X_train = X_train[valid_mask].fillna(0)
    y_train = y_train[valid_mask]
    
    print(f"📊 학습 데이터: {len(X_train)}개 샘플 (NaN/0 이하 제거 후)")
    print(f"   y_train 범위: {y_train.min():.4f} ~ {y_train.max():.4f}")
    print(f"   y_train 평균: {y_train.mean():.4f}, 중앙값: {y_train.median():.4f}")

    # RandomForest 학습 - 최적화된 하이퍼파라미터
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=5,      # 과적합 방지
        min_samples_leaf=2,       # 과적합 방지
        max_features='sqrt',      # 피처 샘플링으로 다양성 증가
        random_state=random_state,
        n_jobs=-1,
        verbose=0
    )
    model.fit(X_train, y_train)
    
    # 피처 중요도 출력 (상위 10개)
    if len(feature_cols) > 0 and len(X_train) > 0:
        try:
            feature_importance = pd.DataFrame({
                'feature': feature_cols,
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            print(f"📊 Top 10 중요 피처:")
            print(feature_importance.head(10).to_string(index=False))
        except:
            pass

    return model, feature_cols



NameError: name 'RandomForestRegressor' is not defined

**코드 셀 동작 설명**

- **주요 범주**: 시각화/플롯 생성, 라이브러리 임포트, 시각화(Seaborn)
- 그래프/차트로 결과 시각화

In [ ]:
# HS4 분포 시각화
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(14,6))
sns.countplot(data=df, x="hs4", order=df["hs4"].value_counts().index)
plt.xticks(rotation=90)
plt.title("HS4 Distribution")
plt.show()

**코드 셀 동작 설명**

- **주요 범주**: 시각화/플롯 생성, 시각화(Seaborn)
- 그래프/차트로 결과 시각화

In [ ]:
# item_id 분포 시각화

plt.figure(figsize=(18,6))
sns.countplot(data=df, x="item_id", order=df["item_id"].value_counts().index[:30])
plt.xticks(rotation=90)
plt.title("Top 30 Item ID Count Distribution")
plt.show()

## 3. item_id Micro-level Deep Dive

- item_id 5~10개 랜덤 추출
- 월별 value / weight / quantity 시계열 패턴 분석
- zero 패턴(0 비율) 분석
- 결측 패턴(NaN 비율) 분석
- 급증/급감(anomaly) 탐지
- HS 코드 cross-check (item_id에 여러 HS4가 존재하는지)
- 각 item_id별 상세 그래프 자동 생성(멀티 플롯)

**코드 셀 동작 설명**

- **주요 범주**: 라이브러리 임포트, 집계/그룹 연산, 수치/데이터프레임 처리
- 그룹 기준으로 집계/통계 계산

In [ ]:
import numpy as np

# df_panel 기준 사용
df = df_panel.copy()

# 8개 랜덤 sample
sample_items = np.random.choice(df['item_id'].unique(), size=8, replace=False)
sample_df = df[df['item_id'].isin(sample_items)]

monthly_trend = (
    sample_df.groupby(['item_id', 'date'])
             [['value', 'weight', 'quantity']]
             .sum()
             .reset_index()
)

monthly_trend.head()

**코드 셀 동작 설명**

- **주요 범주**: 시각화/플롯 생성
- 그래프/차트로 결과 시각화

In [ ]:
# 8개 item value 변화 시계열 그래프
plt.figure(figsize=(14,6))
for item in sample_items:
    tmp = monthly_trend[monthly_trend['item_id'] == item]
    plt.plot(tmp['date'], tmp['value'], label=item)

plt.title("Monthly Value Trend by Sample Item ID")
plt.legend()
plt.show()

monthly_trend[monthly_trend['item_id'] == 'GKQIJYDH']

**코드 셀 동작 설명**

- **주요 범주**: 중복 데이터 처리
- 중복 행을 탐지하여 제거

In [ ]:
df.head()
df.duplicated().sum()

**코드 셀 동작 설명**

- **주요 범주**: 집계/그룹 연산
- 그룹 기준으로 집계/통계 계산

In [ ]:
# Zero / Missing 패턴 분석
# 최소 수정: 필요한 flag 컬럼이 없으면 즉석 생성, sample_items 미정의 시 간단히 정의
if 'sample_items' not in globals():
    sample_items = df['item_id'].dropna().unique()[:8]

flag_sources = [
    ('value_zero', 'value'),
    ('weight_zero', 'weight'),
    ('quantity_zero', 'quantity'),
]
for flag, base in flag_sources:
    if flag not in df.columns and base in df.columns:
        df[flag] = (df[base] == 0).astype(int)

missing_sources = [
    ('value_missing', 'value'),
    ('weight_missing', 'weight'),
    ('quantity_missing', 'quantity'),
]
for flag, base in missing_sources:
    if flag not in df.columns and base in df.columns:
        df[flag] = df[base].isna().astype(int)

zero_missing_stats = (
    df[df['item_id'].isin(sample_items)]
    .groupby('item_id')[['value_zero', 'weight_zero', 'quantity_zero',
                         'value_missing', 'weight_missing', 'quantity_missing']]
    .mean(numeric_only=True)
    .reset_index()
)

zero_missing_stats

**코드 셀 동작 설명**

- **주요 범주**: 집계/그룹 연산, 정렬
- 그룹 기준으로 집계/통계 계산

In [ ]:
#급증/급감 패턴(anomaly) 탐지
monthly_trend = monthly_trend.sort_values(['item_id', 'date'])

# 전월 대비 변화율
monthly_trend['value_pct_change'] = (
    monthly_trend.groupby('item_id')['value'].pct_change()
)

# 급증 top 5, 급감 bottom 5
top_spikes = monthly_trend.sort_values('value_pct_change', ascending=False).head(5)
bottom_spikes = monthly_trend.sort_values('value_pct_change', ascending=True).head(5)

top_spikes, bottom_spikes

**코드 셀 동작 설명**

- **주요 범주**: 집계/그룹 연산
- 그룹 기준으로 집계/통계 계산

In [ ]:
# HS 코드 cross-check (item_id -> hs4 mapping consistency)
item_hs_check = (
    df.groupby('item_id')['hs4']
      .nunique()
      .reset_index(name='hs4_count')
)

# HS4가 여러 개 존재하는 item만 추출
item_hs_issue = item_hs_check[item_hs_check['hs4_count'] > 1]

item_hs_issue

**코드 셀 동작 설명**

- **주요 범주**: 중복 데이터 처리
- 중복 행을 탐지하여 제거

In [ ]:
df.duplicated().sum()

**코드 셀 동작 설명**

- **주요 범주**: 중복 데이터 처리, 행/열 선택 및 필터링
- 중복 행을 탐지하여 제거

In [ ]:
(item_hs_check['hs4_count'] > 1).sum()
df[df['item_id'] == df['item_id'].iloc[0]][['item_id','hs4']].drop_duplicates()

**코드 셀 동작 설명**

- **주요 범주**: 시각화/플롯 생성
- 그래프/차트로 결과 시각화

In [ ]:
# 각 item_id별 상세 그래프 자동 생성 (value/weight/quantity)
for item in sample_items:
    tmp = monthly_trend[monthly_trend['item_id'] == item]

    fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

    axes[0].plot(tmp['date'], tmp['value'])
    axes[0].set_title(f"{item} - Value Trend")

    axes[1].plot(tmp['date'], tmp['weight'])
    axes[1].set_title(f"{item} - Weight Trend")

    axes[2].plot(tmp['date'], tmp['quantity'])
    axes[2].set_title(f"{item} - Quantity Trend")

    plt.tight_layout()
    plt.show()

## 공행성 후보쌍 생성 파이프라인 코드 (v1)

In [ ]:
import numpy as np
import pandas as pd

# ----------------------------------------------------
# 0. 기본 준비: df_panel에서 log_value 피벗 만들기
# ----------------------------------------------------
def build_value_matrix(df_panel: pd.DataFrame) -> pd.DataFrame:
    """
    df_panel: (item_id, date, value ...)가 있는 패널 데이터프레임
    return: index=date, columns=item_id, values=log1p(value)인 매트릭스
    """
    df = df_panel.copy()

    # 필수 컬럼 체크
    required = {"item_id", "date", "value"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"df_panel에 필요한 컬럼이 없습니다: {missing}")

    # 정렬 및 타입 정리
    df = df.sort_values(["date", "item_id"])
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

    # 음수 value는 0으로 클리핑 (log1p 안정성)
    df["value"] = df["value"].clip(lower=0)

    # log1p 변환 + 비유한/비수치 값 NaN 처리
    df["log_value"] = np.log1p(df["value"].astype(float))
    df.loc[~np.isfinite(df["log_value"]), "log_value"] = np.nan

    # 피벗: date × item_id
    value_mat = (
        df.pivot(index="date", columns="item_id", values="log_value")
          .sort_index()
    )
    return value_mat


# ----------------------------------------------------
# 1. 아이템 필터링 (sparse / low-variance 제거)
# ----------------------------------------------------
def filter_items(value_mat: pd.DataFrame,
                 min_nonzero_months: int = 8,
                 min_std: float = 0.2) -> pd.DataFrame:
    """
    공행성 계산에 쓸 item만 필터링
    - 일정 개수 이상 거래가 있는 아이템
    - log_value 변동성이 너무 작은 아이템 제외
    """
    value_nonzero_cnt = (value_mat > 0).sum(axis=0)
    value_std = value_mat.std(axis=0, skipna=True)

    valid_items = value_mat.columns[
        (value_nonzero_cnt >= min_nonzero_months) &
        (value_std >= min_std)
    ]

    return value_mat[valid_items]


# ----------------------------------------------------
# 2. 마켓 인덱스 제거 → residual matrix
# ----------------------------------------------------
def compute_market_residuals(value_mat: pd.DataFrame) -> pd.DataFrame:
    """
    전체 시장 value 합계를 마켓 인덱스로 보고,
    각 item의 log_value에서 단순 회귀로 시장 효과를 제거한 residual을 반환
    """
    # 전체 시장 인덱스: 월별 total value (log1p)
    # value_mat는 이미 log_value이므로, 여기서는 그냥 평균/합 중 하나를 써도 됨
    # 여기서는 "모든 아이템 log_value의 평균"을 시장 인덱스로 사용
    market_index = value_mat.mean(axis=1, skipna=True)

    # 공통 기간 align
    common_index = value_mat.index.intersection(market_index.index)
    X = market_index.loc[common_index].values  # shape: (T,)
    Y = value_mat.loc[common_index]

    resid_mat = pd.DataFrame(index=common_index, columns=Y.columns, dtype=float)

    for col in Y.columns:
        y = Y[col].values
        mask = ~np.isnan(y) & ~np.isnan(X)

        if mask.sum() < 5:
            resid_mat[col] = np.nan
            continue

        X_m = X[mask]
        y_m = y[mask]

        # y = a + b * X 최소제곱
        A = np.vstack([np.ones_like(X_m), X_m]).T
        coef, _, _, _ = np.linalg.lstsq(A, y_m, rcond=None)
        a, b = coef

        y_hat = a + b * X
        resid = y - y_hat
        resid_mat[col] = resid

    return resid_mat


# ----------------------------------------------------
# 3. 동시 상관 + 1개월 lead 상관 계산
# ----------------------------------------------------
def compute_correlations(resid_mat: pd.DataFrame):
    """
    resid_mat: index=date, columns=item_id
    return:
      - corr_same: 같은 시점 residual 피어슨 상관
      - corr_lead: i_t vs j_{t+1} 상관 (i가 선행, j가 후행)
    """
    # 평균 0으로 센터링
    resid_centered = resid_mat - resid_mat.mean(axis=0)

    # 동시 상관
    corr_same = resid_centered.corr()

    # 1개월 lead 상관
    resid_t = resid_centered.iloc[:-1, :]   # t
    resid_t1 = resid_centered.iloc[1:, :]   # t+1

    items = resid_centered.columns
    corr_lead = pd.DataFrame(index=items, columns=items, dtype=float)

    for i in items:
        x = resid_t[i].values
        mask_x = ~np.isnan(x)

        for j in items:
            if i == j:
                corr_lead.loc[i, j] = np.nan
                continue

            y = resid_t1[j].values  # j(t+1)
            mask_y = ~np.isnan(y)

            mask = mask_x & mask_y
            if mask.sum() < 5:
                corr_lead.loc[i, j] = np.nan
                continue

            corr = np.corrcoef(x[mask], y[mask])[0, 1]
            corr_lead.loc[i, j] = corr

    return corr_same, corr_lead


# ----------------------------------------------------
# 4. 공행성 후보쌍 선정
# ----------------------------------------------------
def select_candidate_pairs(corr_same: pd.DataFrame,
                           corr_lead: pd.DataFrame,
                           th_same: float = 0.4,
                           th_lead: float = 0.5,
                           min_diff: float = 0.05,
                           top_k: int | None = None) -> pd.DataFrame:
    """
    corr_same: i,j 동시 상관
    corr_lead: i_t vs j_{t+1} (i 선행, j 후행)
    규칙:
      - corr_same >= th_same
      - corr_lead(i->j)와 corr_lead(j->i) 중 큰 방향을 선택
      - 방향성 차이가 min_diff 이상이고, 그 방향의 corr_lead >= th_lead
    """
    items = corr_same.index
    pairs = []

    for i in items:
        for j in items:
            if i == j:
                continue

            same = corr_same.loc[i, j]
            if pd.isna(same) or same < th_same:
                continue

            lead_ij = corr_lead.loc[i, j]  # i→j
            lead_ji = corr_lead.loc[j, i]  # j→i

            # 둘 다 NaN이면 스킵
            if pd.isna(lead_ij) and pd.isna(lead_ji):
                continue

            # i→j 방향이 더 유력
            if pd.notna(lead_ij):
                if pd.isna(lead_ji) or (lead_ij - lead_ji) >= min_diff:
                    if lead_ij >= th_lead:
                        pairs.append({
                            "leading_item_id": i,
                            "following_item_id": j,
                            "corr_same": same,
                            "corr_lead": lead_ij,
                            "direction": "i_leads_j"
                        })

            # j→i 방향이 더 유력
            if pd.notna(lead_ji):
                if pd.isna(lead_ij) or (lead_ji - lead_ij) >= min_diff:
                    if lead_ji >= th_lead:
                        pairs.append({
                            "leading_item_id": j,
                            "following_item_id": i,
                            "corr_same": same,
                            "corr_lead": lead_ji,
                            "direction": "j_leads_i"
                        })

    candidates_df = pd.DataFrame(pairs)

    if candidates_df.empty:
        return candidates_df

    candidates_df = candidates_df.sort_values(
        ["corr_lead", "corr_same"], ascending=False
    ).drop_duplicates(
        subset=["leading_item_id", "following_item_id"]
    )

    if top_k is not None:
        candidates_df = candidates_df.head(top_k)

    return candidates_df


# ----------------------------------------------------
# 5. 전체 파이프라인 함수
# ----------------------------------------------------
def generate_comovement_candidates(df_panel: pd.DataFrame,
                                   min_nonzero_months: int = 8,
                                   min_std: float = 0.2,
                                   th_same: float = 0.4,
                                   th_lead: float = 0.5,
                                   min_diff: float = 0.05,
                                   top_k: int | None = None) -> pd.DataFrame:
    """
    입력: df_panel (item_id, date, value 포함)
    출력: 공행성 후보쌍 DataFrame
           [leading_item_id, following_item_id, corr_same, corr_lead, direction]
    """
    # 0) value matrix
    value_mat = build_value_matrix(df_panel)

    # 1) item filtering
    value_mat_f = filter_items(
        value_mat,
        min_nonzero_months=min_nonzero_months,
        min_std=min_std
    )

    # 2) residuals
    resid_mat = compute_market_residuals(value_mat_f)

    # 3) correlations
    corr_same, corr_lead = compute_correlations(resid_mat)

    # 4) candidate pairs
    candidates_df = select_candidate_pairs(
        corr_same, corr_lead,
        th_same=th_same,
        th_lead=th_lead,
        min_diff=min_diff,
        top_k=top_k
    )

    return candidates_df


In [ ]:
# df_panel이 이미 준비되어 있다고 가정
candidates_df = generate_comovement_candidates(
    df_panel,
    min_nonzero_months=8,
    min_std=0.2,
    th_same=0.4,
    th_lead=0.5,
    min_diff=0.05,
    top_k=500  # 예시: 상위 500쌍만
)

print(candidates_df)
print("후보쌍 개수:", len(candidates_df))


## 1. 시계열 핵심 피처 추가 (log, lag, rolling)

- 지금 make_item_features는 HS4/HS2 롤링, EMA, run-length를 만들어주고 있어.
- 여기에 log_value + lag1~3 + rolling mean 3/6개월 + trend_index/year/month를 추가로 붙여 쓰자.

In [ ]:
import sys
!{sys.executable} -m pip install scikit-learn


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# -------------------------------
# 1) 시계열 핵심 피처 추가
# -------------------------------
def add_ts_core_features(feat_df: pd.DataFrame) -> pd.DataFrame:
    """
    기존 make_item_features 결과(feat_df)에
    - log_value
    - log_value_lag1,2,3
    - log_roll_mean_3,6
    - year, month, trend_index
    를 추가한다.
    """
    df = feat_df.copy()

    # date 정리
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.sort_values(["item_id", "date"])

    # year, month 없으면 생성
    if "year" not in df.columns:
        df["year"] = df["date"].dt.year
    if "month" not in df.columns:
        df["month"] = df["date"].dt.month

    # trend_index: 날짜 순서 인덱스 (전체 공통)
    unique_dates = sorted(df["date"].dropna().unique())
    date_to_trend = {d: i for i, d in enumerate(unique_dates)}
    df["trend_index"] = df["date"].map(date_to_trend)

    # 계절성 피처 (sin/cos 변환)
    df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
    df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

    # log 변환 (value, weight, quantity)
    for col in ["value", "weight", "quantity"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").fillna(0)
            df[col] = df[col].clip(lower=0)
            df[f"log_{col}"] = np.log1p(df[col].astype(float))

    # item별 lag, rolling 통계
    dfs = []
    for item, sub in df.groupby("item_id"):
        sub = sub.sort_values("date").copy()
        
        # 더 많은 lag 피처 (1,2,3,6,12)
        for lag in [1, 2, 3, 6, 12]:
            if "log_value" in sub.columns:
                sub[f"log_value_lag{lag}"] = sub["log_value"].shift(lag)
            if "log_weight" in sub.columns:
                sub[f"log_weight_lag{lag}"] = sub["log_weight"].shift(lag)
        
        # rolling 통계 (mean, std, min, max)
        for window in [3, 6, 12]:
            if "log_value" in sub.columns:
                shifted = sub["log_value"].shift(1)
                sub[f"log_roll_mean_{window}"] = shifted.rolling(window, min_periods=1).mean()
                sub[f"log_roll_std_{window}"] = shifted.rolling(window, min_periods=1).std().fillna(0)
                if window == 3:  # min/max는 3개월만
                    sub[f"log_roll_min_{window}"] = shifted.rolling(window, min_periods=1).min()
                    sub[f"log_roll_max_{window}"] = shifted.rolling(window, min_periods=1).max()
        
        # YoY 변화율 (12개월 전 대비)
        if "log_value" in sub.columns:
            sub["log_value_yoy"] = sub["log_value"] - sub["log_value"].shift(12)
            shifted_12 = sub["log_value"].shift(12)
            sub["log_value_yoy_pct"] = (sub["log_value"] - shifted_12) / (shifted_12 + 1e-8)

        dfs.append(sub)

    df_out = pd.concat(dfs, axis=0).reset_index(drop=True)
    return df_out


## 2. pair 단위 학습 데이터셋 만들기

### 각 row를 이렇게 보도록 만들 거야:

- (leading_item_id, following_item_id, date_t)

X:

- following의 log_value_lag/roll/EMA/HS통계/zero-run 등

- leading의 log_value_lag/roll/EMA/HS통계/zero-run 등

- t의 year, month, trend_index

y:

- following의 log_value at t+1

In [ ]:
# -------------------------------
# 2) pair-level 학습 데이터 생성
# -------------------------------

def _next_month(d: pd.Timestamp) -> pd.Timestamp:
    year = d.year + (1 if d.month == 12 else 0)
    month = 1 if d.month == 12 else d.month + 1
    return pd.Timestamp(year=year, month=month, day=1)


def build_pair_training_data_v2(
    feat_df: pd.DataFrame,
    candidates_df: pd.DataFrame,
    max_train_date: pd.Timestamp,
) -> tuple[pd.DataFrame, pd.Series, list[str], pd.DataFrame]:
    """
    feat_df: add_ts_core_features까지 적용된 피처 패널
    candidates_df: 공행성 후보쌍 DataFrame
    max_train_date: 학습에 사용할 마지막 t (라벨은 t+1 기준)

    반환:
      X: feature DataFrame
      y: Series (following의 log_value at t+1)
      feature_cols: 사용된 컬럼 리스트
      meta_df: (leading, following, date_t, date_t1) 메타 정보
    """
    df = feat_df.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    base = df.set_index(["item_id", "date"])

    # 사용할 날짜들 (t)
    all_dates = sorted(df["date"].dropna().unique())
    all_dates = [d for d in all_dates if d <= max_train_date]

    # 어떤 피처들을 쓸지 정의
    # (존재 여부를 체크하면서 동적으로 선택)
    candidate_item_features = [
        "value",
        "log_value",
        "log_value_lag1", "log_value_lag2", "log_value_lag3",
        "log_roll_mean_3", "log_roll_mean_6",
        # EMA
        "value_ema_3", "value_ema_6", "value_ema_12",
        # HS 그룹 통계
        "hs4_value_mean_3m", "hs4_value_mean_6m", "hs4_value_mean_12m",
        "hs2_value_mean_3m", "hs2_value_mean_6m", "hs2_value_mean_12m",
        # zero/missing run-length
        "value_zero_run_length",
        "value_missing_run_length",
        "is_missing_month_run_length",
    ]
    # 실제로 feat_df에 존재하는 것만 사용
    item_features = [c for c in candidate_item_features if c in df.columns]

    rows = []

    for _, pair in candidates_df.iterrows():
        lead = pair["leading_item_id"]
        foll = pair["following_item_id"]

        for t in all_dates:
            t1 = _next_month(t)

            # t+1 레이블이 없으면 스킵
            if (foll, t1) not in base.index:
                continue

            # t 시점에 leading/following 피처가 모두 있어야 함
            if (foll, t) not in base.index or (lead, t) not in base.index:
                continue

            foll_t = base.loc[(foll, t)]
            lead_t = base.loc[(lead, t)]
            foll_t1 = base.loc[(foll, t1)]

            row = {
                "leading_item_id": lead,
                "following_item_id": foll,
                "date_t": t,
                "date_t1": t1,
                # 시점 공통 피처
                "year": foll_t.get("year", np.nan),
                "month": foll_t.get("month", np.nan),
                "trend_index": foll_t.get("trend_index", np.nan),
                # 타겟 (log_value at t+1)
                "y_log_next": foll_t1.get("log_value", np.nan),
            }

            # following / leading 각각에 대해 item_features 추가
            for c in item_features:
                row[f"f_{c}"] = foll_t.get(c, np.nan)
                row[f"l_{c}"] = lead_t.get(c, np.nan)

            rows.append(row)

    pair_df = pd.DataFrame(rows)
    # NaN 많은 초기 구간 제거
    pair_df = pair_df.dropna(subset=["y_log_next"]).reset_index(drop=True)

    # feature 컬럼 목록
    feature_cols = ["year", "month", "trend_index"]
    feature_cols += [col for col in pair_df.columns if col.startswith("f_")]
    feature_cols += [col for col in pair_df.columns if col.startswith("l_")]

    X = pair_df[feature_cols].copy()
    y = pair_df["y_log_next"].copy()

    return X, y, feature_cols, pair_df


## 3. RandomForest로 학습 + 검증 (time-split)

- train_last_t까지의 t를 train

- valid_last_t까지의 t를 한 번 더 써서 valid 구간 생성
(valid는 train_last_t 이후 t에 해당하는 row만 남김)

In [ ]:
# ----------------------------------------------------
# 아이템 단위 피처 생성 함수 (옵션형 블록)
# ----------------------------------------------------

def _compute_run_length(flag_series: pd.Series) -> pd.Series:
    """0/1 플래그 시리즈에서 연속 1(run length)을 계산.
    1 구간은 누적 길이, 0 구간은 0으로 둔다.
    """
    flag = flag_series.fillna(0).astype(int)
    is_one = flag == 1
    # 0이 나올 때마다 그룹이 증가 → 그룹별로 cumsum하면 run length
    groups = (~is_one).cumsum()
    run = is_one.groupby(groups).cumsum()
    return run.where(is_one, 0)


def make_item_features(
    df_panel: pd.DataFrame,
    use_hs_stats: bool = True,
    use_ema: bool = True,
    use_zero_run: bool = True,
    hs_windows: tuple[int, ...] = (3, 6, 12),
    ema_spans: tuple[int, ...] = (3, 6, 12),
) -> pd.DataFrame:
    """df_panel에서 아이템 단위 시계열 피처를 생성.

    옵션 플래그로 HS 그룹 롤링 통계 / EMA / zero-run-length 블록을 온오프 할 수 있다.
    반환값은 (item_id, date)를 유지한 행 단위 피처 DataFrame.
    """
    df = df_panel.copy()

    # 기본 정렬 및 타입 정리
    if "date" not in df.columns or "item_id" not in df.columns:
        raise ValueError("df_panel에는 최소한 'item_id', 'date' 컬럼이 필요합니다.")

    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.sort_values(["item_id", "date"])  # 아이템별 시간 순 정렬

    # value/weight/quantity를 수치형으로 강제
    for col in ["value", "weight", "quantity"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # ------------------------------------------------
    # 1) HS4 / HS2 그룹 롤링 통계 (옵션)
    # ------------------------------------------------
    if use_hs_stats:
        # HS4 기준 롤링
        if "hs4" in df.columns:
            for w in hs_windows:
                # HS4 그룹별 월별 value 평균 롤링
                col_name = f"hs4_value_mean_{w}m"
                df[col_name] = (
                    df.groupby("hs4")["value"]
                      .transform(lambda s: s.rolling(window=w, min_periods=1).mean())
                )
        # HS2가 있으면 추가 롤링 통계
        if "hs2" in df.columns:
            for w in hs_windows:
                col_name = f"hs2_value_mean_{w}m"
                df[col_name] = (
                    df.groupby("hs2")["value"]
                      .transform(lambda s: s.rolling(window=w, min_periods=1).mean())
                )

    # ------------------------------------------------
    # 2) EMA 피처 (옵션)
    # ------------------------------------------------
    if use_ema:
        for span in ema_spans:
            if "value" in df.columns:
                df[f"value_ema_{span}"] = (
                    df.groupby("item_id")["value"]
                      .transform(lambda s: s.ewm(span=span, adjust=False).mean())
                )
            if "weight" in df.columns:
                df[f"weight_ema_{span}"] = (
                    df.groupby("item_id")["weight"]
                      .transform(lambda s: s.ewm(span=span, adjust=False).mean())
                )
            if "quantity" in df.columns:
                df[f"quantity_ema_{span}"] = (
                    df.groupby("item_id")["quantity"]
                      .transform(lambda s: s.ewm(span=span, adjust=False).mean())
                )

    # ------------------------------------------------
    # 3) zero / missing run-length 피처 (옵션)
    # ------------------------------------------------
    if use_zero_run:
        # 기대하는 플래그 컬럼: value_zero, value_missing, is_missing_month 등
        flag_cols = [
            col for col in [
                "value_zero",
                "weight_zero",
                "quantity_zero",
                "value_missing",
                "weight_missing",
                "quantity_missing",
                "is_missing_month",
            ]
            if col in df.columns
        ]

        for col in flag_cols:
            feat_name = f"{col}_run_length"
            df[feat_name] = (
                df.groupby("item_id")[col]
                  .transform(_compute_run_length)
            )

    # 최종 피처 DataFrame: 원본 index/키 유지
    return df


## 4. 2025-08 pair-level 예측

- t = 2025-07-01를 기준으로 feature를 만들고

- each pair(leading, following)에 대해 X를 만들고

- log-space 예측 후 expm1 → 정수로 클립

In [ ]:
# -------------------------------
# 4) 2025-08 예측용 pair-level 데이터 생성
# -------------------------------

def build_pair_features_for_target_month(
    feat_df: pd.DataFrame,
    candidates_df: pd.DataFrame,
    target_prev_month: pd.Timestamp,
    feature_cols: list[str],
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    target_prev_month = t (예: 2025-07-01)
    t 시점 feature로 t+1(2025-08) value를 예측하기 위한 X를 만든다.
    """
    df = feat_df.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    base = df.set_index(["item_id", "date"])

    item_features = [c for c in feature_cols if c.startswith("f_")]
    # f_, l_ 패턴과 year, month, trend_index만 사용

    rows = []

    # 입력 검증
    if candidates_df is None or len(candidates_df) == 0:
        return pd.DataFrame(columns=["leading_item_id", "following_item_id"]), pd.DataFrame()
    
    if "leading_item_id" not in candidates_df.columns or "following_item_id" not in candidates_df.columns:
        raise ValueError("candidates_df에 'leading_item_id' 또는 'following_item_id' 컬럼이 없습니다!")
    
    # 필수 컬럼 확인
    if "item_id" not in df.columns or "date" not in df.columns:
        raise ValueError("feat_df에 'item_id' 또는 'date' 컬럼이 없습니다!")
    
    try:
        base = df.set_index(["item_id", "date"])
    except Exception as e:
        raise ValueError(f"인덱스 설정 실패: {e}")

    for _, pair in candidates_df.iterrows():
        try:
            lead = pair["leading_item_id"]
            foll = pair["following_item_id"]
            t = target_prev_month

            # None이나 NaN 체크
            if pd.isna(lead) or pd.isna(foll):
                continue

            if (foll, t) not in base.index or (lead, t) not in base.index:
                continue

            foll_t = base.loc[(foll, t)]
            lead_t = base.loc[(lead, t)]

            # Series나 dict 모두 처리하는 헬퍼 함수
            def safe_get(data, key, default=np.nan):
                if hasattr(data, 'get'):
                    return data.get(key, default)
                elif hasattr(data, '__getitem__'):
                    try:
                        return data[key]
                    except (KeyError, IndexError):
                        return default
                else:
                    return default

            row = {
                "leading_item_id": lead,
                "following_item_id": foll,
                "date_t": t,
                "year": safe_get(foll_t, "year"),
                "month": safe_get(foll_t, "month"),
                "trend_index": safe_get(foll_t, "trend_index"),
            }

            # feature_cols를 재활용해 동일한 컬럼 구조를 맞춘다.
            # feature_cols 안에 있는 f_/l_ 이름을 기준으로 값 채우기
            for col in feature_cols:
                try:
                    if col.startswith("f_"):
                        base_name = col[2:]
                        row[col] = safe_get(foll_t, base_name)
                    elif col.startswith("l_"):
                        base_name = col[2:]
                        row[col] = safe_get(lead_t, base_name)
                    elif col in ["year", "month", "trend_index", "month_sin", "month_cos"]:
                        # 공유 피처는 following 기준으로 사용
                        if col not in row:
                            row[col] = safe_get(foll_t, col)
                except Exception:
                    row[col] = np.nan

            rows.append(row)
        except Exception as e:
            # 개별 쌍 처리 중 오류는 무시하고 계속
            continue

    if len(rows) == 0:
        return pd.DataFrame(columns=["leading_item_id", "following_item_id"]), pd.DataFrame()

    feat_pair_df = pd.DataFrame(rows)
    
    # feature_cols가 없으면 빈 DataFrame 반환
    if len(feature_cols) == 0:
        return feat_pair_df, pd.DataFrame()
    
    # 존재하는 feature_cols만 사용
    available_cols = [c for c in feature_cols if c in feat_pair_df.columns]
    if len(available_cols) == 0:
        # feature_cols가 없으면 기본 컬럼만 반환
        X_target = pd.DataFrame(index=feat_pair_df.index)
    else:
        X_target = feat_pair_df[available_cols].copy()

    return feat_pair_df, X_target


def predict_aug_2025_values_v2(
    model,
    feat_df: pd.DataFrame,
    candidates_df: pd.DataFrame,
    feature_cols: list[str],
    submission_template: pd.DataFrame | None = None,
) -> pd.DataFrame:
    """
    pair-level RF 모델로 각 (leading, following) 쌍의 2025-08 value 예측
    
    Args:
        model: 학습된 모델
        feat_df: 피처 데이터프레임
        candidates_df: 공행성 후보쌍 (기본 예측 대상)
        feature_cols: 피처 컬럼 리스트
        submission_template: 제출 템플릿 (있으면 모든 쌍에 대해 예측 시도)
    """
    # t = 2025-07-01 → t+1 = 2025-08-01
    last_obs_date = feat_df["date"].max()
    target_prev_month = last_obs_date  # df_panel이 2025-07-01까지 있다고 가정

    # submission template이 있으면 모든 쌍에 대해 예측 시도
    if submission_template is not None and len(submission_template) > 0:
        try:
            # template의 모든 쌍을 candidates로 사용
            all_pairs = submission_template[["leading_item_id", "following_item_id"]].copy()
            # 중복 제거
            all_pairs = all_pairs.drop_duplicates()
            print(f"📊 submission template의 {len(all_pairs)}개 쌍에 대해 예측 시도")
        except Exception as e:
            print(f"⚠️ submission_template 처리 중 오류: {e}, candidates_df 사용")
            all_pairs = candidates_df[["leading_item_id", "following_item_id"]].copy()
            print(f"📊 candidates_df의 {len(all_pairs)}개 쌍에 대해 예측 시도")
    else:
        if candidates_df is None or len(candidates_df) == 0:
            raise ValueError("candidates_df가 비어있습니다!")
        all_pairs = candidates_df[["leading_item_id", "following_item_id"]].copy()
        print(f"📊 candidates_df의 {len(all_pairs)}개 쌍에 대해 예측 시도")

    if len(all_pairs) == 0:
        raise ValueError("예측할 쌍이 없습니다!")

    try:
        feat_pair_df, X_target = build_pair_features_for_target_month(
            feat_df, all_pairs, target_prev_month, feature_cols
        )
    except Exception as e:
        print(f"⚠️ 피처 생성 중 오류: {e}")
        raise

    if len(feat_pair_df) == 0:
        print("⚠️ 예측 가능한 쌍이 없습니다. 빈 결과 반환")
        return pd.DataFrame(columns=["leading_item_id", "following_item_id", "value"])

    try:
        # NaN 처리 후 예측
        X_target_filled = X_target.fillna(0)
        y_pred_log = model.predict(X_target_filled)
        
        # 예측값 검증
        print(f"   예측 log_value 범위: {y_pred_log.min():.4f} ~ {y_pred_log.max():.4f}")
        print(f"   예측 log_value 평균: {y_pred_log.mean():.4f}")
        
        # expm1 변환: log(1+x) → x
        y_pred_value = np.expm1(y_pred_log)
        
        # 음수나 이상치 처리
        y_pred_value = np.clip(y_pred_value, 0, None)
        
        # 이상치 제거 (99.9 percentile)
        if len(y_pred_value) > 0:
            upper_bound = np.percentile(y_pred_value, 99.9)
            y_pred_value = np.clip(y_pred_value, 0, upper_bound)
            print(f"   변환된 value 범위: {y_pred_value.min():.2f} ~ {y_pred_value.max():.2f}")
            print(f"   변환된 value 평균: {y_pred_value.mean():.2f}, 중앙값: {np.median(y_pred_value):.2f}")
        
    except Exception as e:
        print(f"❌ 예측 중 오류: {e}")
        raise

    result = feat_pair_df[["leading_item_id", "following_item_id"]].copy()
    result["value"] = y_pred_value
    result["value"] = result["value"].clip(lower=0).round().astype(int)
    
    # 최소값 보장 (0이 너무 많으면 문제)
    zero_count = (result["value"] == 0).sum()
    if zero_count > len(result) * 0.1:  # 10% 이상이 0이면
        non_zero_values = result[result["value"] > 0]["value"]
        if len(non_zero_values) > 0:
            non_zero_mean = non_zero_values.mean()
            min_val = max(1, int(non_zero_mean * 0.01))
            result.loc[result["value"] == 0, "value"] = min_val
            print(f"   ⚠️ {zero_count}개 0 값을 최소값({min_val})으로 대체")

    print(f"✅ {len(result)}개 쌍 예측 완료 (평균값: {result['value'].mean():.2f}, 중앙값: {result['value'].median():.2f})")
    
    return result


## 5. 제출용 DataFrame 만들기

- 이미 sample_submission.csv를 쓴 구조가 있으니,

- 이번엔 pair 예측 결과를 sample_submission의 (leading, following)와 매칭해서 value를 채우는 방식으로 갈 수 있어.

- 지금 너가 쓰던 make_submission_df를 v2용으로 약간 바꾸면:

In [ ]:
def make_submission_from_pair_pred(
    pair_pred_df: pd.DataFrame,
    submission_template: pd.DataFrame | None = None,
    feat_df: pd.DataFrame | None = None,
) -> pd.DataFrame:
    """
    pair_pred_df: (leading_item_id, following_item_id, value)
    submission_template: sample_submission (optional)
    feat_df: 피처 데이터프레임 (fallback을 위한 아이템별 평균 계산용)
    """
    # 입력 검증
    if pair_pred_df is None or len(pair_pred_df) == 0:
        raise ValueError("pair_pred_df가 비어있습니다!")
    
    if "value" not in pair_pred_df.columns:
        raise ValueError("pair_pred_df에 'value' 컬럼이 없습니다!")
    
    if submission_template is None:
        # 후보쌍만 제출하는 경우
        required_cols = ["leading_item_id", "following_item_id", "value"]
        missing_cols = [c for c in required_cols if c not in pair_pred_df.columns]
        if missing_cols:
            raise ValueError(f"pair_pred_df에 필요한 컬럼이 없습니다: {missing_cols}")
        
        sub = pair_pred_df[required_cols].copy()
        sub = sub.drop_duplicates()
        return sub

    # template 기준으로 채우는 경우
    if "leading_item_id" not in submission_template.columns or "following_item_id" not in submission_template.columns:
        raise ValueError("submission_template에 'leading_item_id' 또는 'following_item_id' 컬럼이 없습니다!")
    
    base = submission_template[["leading_item_id", "following_item_id"]].copy()
    
    # pair_pred_df에서 value_map 생성
    if len(pair_pred_df) > 0:
        key = list(zip(pair_pred_df["leading_item_id"], pair_pred_df["following_item_id"]))
        val = pair_pred_df["value"].values
        value_map = dict(zip(key, val))
    else:
        value_map = {}
        print("⚠️ pair_pred_df가 비어있어 모든 값에 fallback 사용")

    base_key = list(zip(base["leading_item_id"], base["following_item_id"]))
    
    # 예측값이 없는 쌍에 대한 fallback 전략 (개선)
    if feat_df is not None and "value" in feat_df.columns and len(feat_df) > 0:
        try:
            # 최근 6개월 데이터만 사용하여 더 정확한 평균 계산
            recent_date = feat_df["date"].max()
            recent_cutoff = recent_date - pd.DateOffset(months=6)
            recent_feat_df = feat_df[feat_df["date"] >= recent_cutoff]
            
            if len(recent_feat_df) > 0:
                # 아이템별 최근 평균값
                item_means = recent_feat_df.groupby("item_id")["value"].mean().to_dict()
                # 아이템별 최근 중앙값 (더 robust)
                item_medians = recent_feat_df.groupby("item_id")["value"].median().to_dict()
                global_mean = float(recent_feat_df["value"].mean())
                global_median = float(recent_feat_df["value"].median())
            else:
                # 최근 데이터가 없으면 전체 데이터 사용
                item_means = feat_df.groupby("item_id")["value"].mean().to_dict()
                item_medians = feat_df.groupby("item_id")["value"].median().to_dict()
                global_mean = float(feat_df["value"].mean())
                global_median = float(feat_df["value"].median())
            
            # 예측값이 없으면 following_item_id의 중앙값 사용 (평균보다 robust)
            base["value"] = [
                value_map.get(k, item_medians.get(k[1], item_means.get(k[1], global_median))) 
                for k in base_key
            ]
            missing_count = sum(1 for k in base_key if k not in value_map)
            if missing_count > 0:
                print(f"⚠️ {missing_count}개 쌍에 대해 예측값이 없어 fallback 사용 (아이템별 중앙값/평균 또는 전체 중앙값)")
        except Exception as e:
            print(f"⚠️ fallback 계산 중 오류: {e}, 예측값 중앙값 사용")
            pred_median = float(pair_pred_df["value"].median()) if len(pair_pred_df) > 0 and "value" in pair_pred_df.columns else 1.0
            base["value"] = [value_map.get(k, pred_median) for k in base_key]
    else:
        # feat_df가 없으면 예측값의 평균 사용
        try:
            pred_mean = float(pair_pred_df["value"].mean()) if len(pair_pred_df) > 0 and "value" in pair_pred_df.columns else 1.0
        except:
            pred_mean = 1.0
        
        base["value"] = [value_map.get(k, pred_mean) for k in base_key]
        missing_count = sum(1 for k in base_key if k not in value_map)
        if missing_count > 0:
            print(f"⚠️ {missing_count}개 쌍에 대해 예측값이 없어 평균값({pred_mean:.2f}) 사용")

    return base


In [ ]:
# 필요한 import 확인
import os
import numpy as np
import pandas as pd

# 필요한 변수들이 정의되어 있는지 확인
required_vars = ['df_panel', 'candidates_df']
missing_vars = [var for var in required_vars if var not in globals()]

if missing_vars:
    raise NameError(
        f"다음 변수들이 정의되지 않았습니다: {', '.join(missing_vars)}\n"
        f"다음 셀들을 먼저 실행해주세요:\n"
        f"- df_panel: 패널 데이터 생성 셀 (보통 전처리 섹션)\n"
        f"- candidates_df: 공행성 후보쌍 생성 셀 (generate_comovement_candidates 함수 실행)"
    )

# 변수 검증
print("📊 변수 확인:")
print(f"   - df_panel: {len(df_panel) if df_panel is not None else 'None'} rows")
print(f"   - candidates_df: {len(candidates_df) if candidates_df is not None else 'None'} pairs")

# 1. 아이템 피처 생성 (기존 함수 + 시계열 코어 피처)
print("\n🔧 피처 생성 중...")
try:
    feat_base = make_item_features(
        df_panel,
        use_hs_stats=True,
        use_ema=True,
        use_zero_run=True
    )          # HS/EMA/run-length
    print(f"   ✅ 기본 피처 생성 완료: {len(feat_base)} rows, {len(feat_base.columns)} columns")
except Exception as e:
    print(f"   ⚠️ 기본 피처 생성 중 오류: {e}")
    # 파라미터 없이 시도
    feat_base = make_item_features(df_panel)
    print(f"   ✅ 기본 피처 생성 완료 (기본 파라미터): {len(feat_base)} rows")

try:
    feat_df = add_ts_core_features(feat_base)         # log/lag/rolling/year/month/trend_index
    print(f"   ✅ 시계열 코어 피처 추가 완료: {len(feat_df)} rows, {len(feat_df.columns)} columns")
except Exception as e:
    print(f"   ❌ 시계열 코어 피처 추가 실패: {e}")
    raise

# 2. 모델 학습 (pair-level RF) - 최적화된 하이퍼파라미터 사용
train_last_t = pd.Timestamp("2024-12-01")
valid_last_t = pd.Timestamp("2025-06-01")

print("\n🔧 모델 학습 시작...")
print(f"   - 학습 기간: ~{train_last_t}")
print(f"   - 검증 기간: ~{valid_last_t}")
print(f"   - 후보쌍 수: {len(candidates_df)}")

# 학습 데이터 생성 가능 여부 사전 확인
feat_dates = pd.to_datetime(feat_df["date"], errors="coerce")
available_dates = sorted(feat_dates.dropna().unique())
train_dates = [d for d in available_dates if d <= train_last_t]

print(f"   - 사용 가능한 날짜 수: {len(available_dates)}")
print(f"   - 학습 기간 내 날짜 수: {len(train_dates)}")

if len(train_dates) < 2:
    print("⚠️ 경고: 학습에 사용할 충분한 날짜가 없습니다!")
    print(f"   사용 가능한 날짜: {available_dates[:5]}... (최대 {len(available_dates)}개)")

try:
    rf_model, feature_cols = train_pair_model_v2(
        feat_df,
        candidates_df,
        train_last_t=train_last_t,
        valid_last_t=valid_last_t,
        n_estimators=300,  # 더 많은 트리로 성능 향상
        max_depth=20,      # 더 깊은 트리로 복잡한 패턴 학습
        random_state=42,
    )
    print(f"✅ 모델 학습 완료! 사용된 피처 수: {len(feature_cols)}")
except ValueError as e:
    print(f"❌ 모델 학습 실패: {e}")
    print("\n🔍 자동으로 train_last_t 조정 시도...")
    
    # train_last_t를 점진적으로 조정하여 학습 데이터 생성 시도
    adjusted_train_last_t = train_last_t
    success = False
    
    for months_back in [1, 2, 3, 6, 12]:
        adjusted_train_last_t = train_last_t - pd.DateOffset(months=months_back)
        adjusted_train_dates = [d for d in available_dates if d <= adjusted_train_last_t]
        
        if len(adjusted_train_dates) >= 2:
            print(f"   - {months_back}개월 전으로 조정: {adjusted_train_last_t}")
            try:
                rf_model, feature_cols = train_pair_model_v2(
                    feat_df,
                    candidates_df,
                    train_last_t=adjusted_train_last_t,
                    valid_last_t=valid_last_t,
                    n_estimators=300,
                    max_depth=20,
                    random_state=42,
                )
                print(f"✅ 모델 학습 완료! (조정된 학습 기간 사용)")
                print(f"   사용된 피처 수: {len(feature_cols)}")
                success = True
                break
            except ValueError:
                continue
    
    if not success:
        print("\n❌ 자동 조정 실패. 다음을 확인하세요:")
        print("   1. candidates_df에 유효한 쌍이 있는지 확인")
        print("   2. feat_df에 충분한 날짜 데이터가 있는지 확인")
        print("   3. candidates_df의 item_id가 feat_df에 존재하는지 확인")
        raise ValueError("학습 데이터를 생성할 수 없습니다. 데이터를 확인하세요.")

# 3. sample_submission 로드 (예측 전에 로드)
sample_path = os.path.join("..", "sample_submission.csv")
if os.path.exists(sample_path):
    submission_template = pd.read_csv(sample_path)
    print(f"✅ submission template 로드 완료: {len(submission_template)}개 쌍")
else:
    submission_template = None
    print("[WARN] sample_submission.csv를 찾을 수 없습니다. 후보쌍만 사용합니다.")

# 4. 2025-08 예측 (pair 단위) - submission template의 모든 쌍에 대해 예측!
print("\n🔮 예측 수행 중...")
try:
    pair_pred_df = predict_aug_2025_values_v2(
        rf_model,
        feat_df,
        candidates_df,
        feature_cols,
        submission_template=submission_template,  # 모든 쌍에 대해 예측 시도
    )
    print(f"   ✅ 예측 완료: {len(pair_pred_df)}개 쌍 예측됨")
except Exception as e:
    print(f"   ❌ 예측 실패: {e}")
    raise

# 5. sample_submission을 기준으로 제출 파일 생성 (fallback 포함)
print("\n📝 Submission 파일 생성 중...")
try:
    submission_df = make_submission_from_pair_pred(
        pair_pred_df,
        submission_template=submission_template,
        feat_df=feat_df,  # fallback을 위한 피처 데이터 전달
    )
    print(f"   ✅ Submission 생성 완료: {len(submission_df)} rows")
except Exception as e:
    print(f"   ❌ Submission 생성 실패: {e}")
    raise

# 6. 파일 저장 및 검증
try:
    submission_df.to_csv("submission_kmu_v2_pair_rf.csv", index=False)
    print(f"\n✅ 파일 저장 완료: submission_kmu_v2_pair_rf.csv")
    print(f"\n📊 Submission 통계:")
    print(f"   - 총 row 수: {len(submission_df):,}")
    print(f"   - 평균값: {submission_df['value'].mean():.2f}")
    print(f"   - 중앙값: {submission_df['value'].median():.2f}")
    print(f"   - 최소값: {submission_df['value'].min():,}")
    print(f"   - 최대값: {submission_df['value'].max():,}")
    print(f"   - 0 값 개수: {(submission_df['value'] == 0).sum():,}")
    print(f"\n처음 10개 행:")
    print(submission_df.head(10))
except Exception as e:
    print(f"   ❌ 파일 저장 실패: {e}")
    raise


In [ ]:
# ----------------------------------------------------
# 간단한 baseline 모델: item별 평균 value 기반 예측
# 외부 라이브러리 의존 없이 사용 가능하도록 설계
# ----------------------------------------------------
'''
class SimpleItemMeanModel:
    """item_id별 평균 value를 사용하는 아주 단순한 회귀 모델.

    predict(X)는 X['item_id']를 사용해 해당 아이템 평균값을 반환하고,
    없는 아이템은 전체 평균(global_mean)을 사용한다.
    """

    def __init__(self, item_means: dict, global_mean: float):
        self.item_means = item_means
        self.global_mean = global_mean

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        if "item_id" not in X.columns:
            raise ValueError("입력 X에는 'item_id' 컬럼이 필요합니다.")
        s = X["item_id"].map(self.item_means).fillna(self.global_mean)
        return s.values


# ----------------------------------------------------
# 3. 모델 학습 함수 (baseline 버전)
# ----------------------------------------------------

def train_value_model(
    feat_df: pd.DataFrame,
    candidates_df: pd.DataFrame,
    train_last_t: pd.Timestamp,
    valid_last_t: pd.Timestamp,
):
    """아이템별 평균 value를 사용하는 간단한 baseline 모델 학습.

    - feat_df: make_item_features로 생성된 피처 DF (item_id, date, value 등 포함)
    - candidates_df: 공행성 후보쌍 (현재 버전에서는 item 필터용으로만 활용 가능)
    - train_last_t: 학습에 사용할 마지막 date (이 날짜 이하만 사용)
    - valid_last_t: 검증용 마지막 date (향후 고급 모델에서 활용 가능; 현재는 미사용)

    반환:
      - model: SimpleItemMeanModel 인스턴스
      - feature_cols: 예측 시 사용할 피처 컬럼 리스트 (여기서는 'item_id'만 사용)
    """
    df = feat_df.copy()

    # date 정리
    if "date" not in df.columns:
        raise ValueError("feat_df에는 'date' 컬럼이 필요합니다.")
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # value 컬럼 확인
    if "value" not in df.columns:
        raise ValueError("feat_df에는 예측 대상인 'value' 컬럼이 필요합니다.")

    # 공행성 후보쌍에 등장하는 item만 학습에 사용하고 싶다면 필터링 (선택적)
    if not candidates_df.empty:
        items_in_pairs = set(candidates_df["leading_item_id"]).union(
            set(candidates_df["following_item_id"])
        )
        df = df[df["item_id"].isin(items_in_pairs)]

    # 학습 구간: train_last_t 이하
    train_mask = df["date"] <= train_last_t
    train_df = df[train_mask].dropna(subset=["value"])

    if train_df.empty:
        raise ValueError("train_value_model: 학습에 사용할 데이터가 없습니다.")

    # item별 평균 및 전체 평균 계산
    item_means = train_df.groupby("item_id")["value"].mean().to_dict()
    global_mean = float(train_df["value"].mean())

    model = SimpleItemMeanModel(item_means, global_mean)
    feature_cols = ["item_id"]  # 이 baseline에서는 item_id만 사용

    return model, feature_cols


# ----------------------------------------------------
# 4. 2025-08 value 예측 함수 (baseline 버전)
# ----------------------------------------------------

def predict_aug_2025_values(
    model: SimpleItemMeanModel,
    feat_df: pd.DataFrame,
    candidates_df: pd.DataFrame,
    feature_cols: list[str],
    items_override: list[str] | None = None,
) -> pd.DataFrame:
    """간단 baseline: 각 item_id에 대해 2025-08-01의 value를 예측.

    `items_override`가 주어지면 해당 목록을 그대로 사용하고,
    없으면 공행성 후보쌍 아이템 또는 전체 item 목록을 따른다.
    (추후 고급 버전에서는 롤링/lag 피처를 사용해 one-step ahead 예측으로 교체 가능)
    """
    df = feat_df.copy()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")

    # 예측 대상 item 목록 결정
    if items_override is not None and len(items_override) > 0:
        items = sorted(pd.Series(items_override).dropna().unique())
    elif not candidates_df.empty:
        items = sorted(
            set(candidates_df["leading_item_id"]).union(
                set(candidates_df["following_item_id"])
            )
        )
    else:
        items = sorted(df["item_id"].dropna().unique())

    target_date = pd.Timestamp("2025-08-01")

    # 예측용 입력 DataFrame (모델은 item_id만 사용)
    X_pred = pd.DataFrame({
        "item_id": items,
        "date": target_date,
    })

    preds = model.predict(X_pred)

    pred_df = X_pred.copy()
    pred_df["pred_value"] = preds

    return pred_df


# ----------------------------------------------------
# 5. 제출용 DataFrame 생성 (간단 버전)
# ----------------------------------------------------

def make_submission_df(
    pred_df: pd.DataFrame,
    submission_template: pd.DataFrame | None = None,
    candidate_pairs: pd.DataFrame | None = None,
) -> pd.DataFrame:
    """예측 결과를 제출 포맷(leading/following/value)으로 정리한다.

    - submission_template: 대회에서 제공한 sample_submission 형태의 DataFrame.
      (leading_item_id, following_item_id 컬럼이 있어야 함)
    - candidate_pairs: 템플릿이 없을 때 사용할 공행성 후보쌍 DataFrame.
    """
    if "pred_value" not in pred_df.columns:
        raise ValueError("pred_df에는 'pred_value' 컬럼이 필요합니다.")

    if submission_template is not None:
        base = submission_template[["leading_item_id", "following_item_id"]].copy()
    elif candidate_pairs is not None and not candidate_pairs.empty:
        base = candidate_pairs[["leading_item_id", "following_item_id"]].copy()
    else:
        raise ValueError("제출 템플릿이나 후보쌍 DataFrame 중 하나는 반드시 필요합니다.")

    value_map = pred_df.set_index("item_id")["pred_value"]
    default_value = float(pred_df["pred_value"].mean())

    base["value"] = base["following_item_id"].map(value_map).fillna(default_value)

    return base
'''

In [ ]:
'''# 2. item feature 생성
feat_df = make_item_features(df_panel)

# 3. 모델 학습
model, feature_cols = train_value_model(
    feat_df,
    candidates_df,
    train_last_t=pd.Timestamp("2024-12-01"),
    valid_last_t=pd.Timestamp("2025-06-01")
)

# 4. 제출 템플릿 정보 로드
submission_template_path = os.path.join("..", "sample_submission.csv")
submission_template = None
submission_following_items: list[str] | None = None

if os.path.exists(submission_template_path):
    submission_template = pd.read_csv(submission_template_path)
    submission_following_items = (
        submission_template["following_item_id"].dropna().unique().tolist()
    )
else:
    print(
        f"[WARN] 제출 템플릿({submission_template_path})을 찾을 수 없어 후보쌍 기반으로 진행합니다."
    )

# 5. 2025-08 예측
pred_df = predict_aug_2025_values(
    model,
    feat_df,
    candidates_df,
    feature_cols,
    items_override=submission_following_items,
)

# 6. 제출 CSV 생성
submission_df = make_submission_df(
    pred_df,
    submission_template=submission_template,
    candidate_pairs=candidates_df,
)
submission_df.to_csv("submission_kmu_v1.csv", index=False)
'''

In [ ]:
# ============================================================
# ✅ 개선된 submission 생성 코드 (pair-level 예측 사용)
# ============================================================
# 이전 코드는 item-level 예측만 사용해서 점수가 낮았습니다.
# 이제 pair-level 예측을 사용하여 (leading_item_id, following_item_id) 쌍별로 예측합니다.

# 필요한 변수들이 정의되어 있는지 확인
required_vars = ['df_panel', 'candidates_df']
missing_vars = [var for var in required_vars if var not in globals()]

if missing_vars:
    raise NameError(
        f"다음 변수들이 정의되지 않았습니다: {', '.join(missing_vars)}\n"
        f"다음 셀들을 먼저 실행해주세요:\n"
        f"- df_panel: 패널 데이터 생성 셀 (보통 전처리 섹션)\n"
        f"- candidates_df: 공행성 후보쌍 생성 셀 (generate_comovement_candidates 함수 실행)"
    )

# 1. 아이템 피처 생성 (기존 함수 + 시계열 코어 피처)
feat_base = make_item_features(df_panel)          # HS/EMA/run-length
feat_df = add_ts_core_features(feat_base)         # log/lag/rolling/year/month/trend_index

# 2. 모델 학습 (pair-level RF) - 최적화된 하이퍼파라미터 사용
train_last_t = pd.Timestamp("2024-12-01")
valid_last_t = pd.Timestamp("2025-06-01")

print("🔧 모델 학습 시작...")
rf_model, feature_cols = train_pair_model_v2(
    feat_df,
    candidates_df,
    train_last_t=train_last_t,
    valid_last_t=valid_last_t,
    n_estimators=300,  # 더 많은 트리로 성능 향상
    max_depth=20,      # 더 깊은 트리로 복잡한 패턴 학습
    random_state=42,
)
print(f"✅ 모델 학습 완료! 사용된 피처 수: {len(feature_cols)}")

# 3. sample_submission 로드 (예측 전에 로드)
sample_path = os.path.join("..", "sample_submission.csv")
if os.path.exists(sample_path):
    submission_template = pd.read_csv(sample_path)
    print(f"✅ submission template 로드 완료: {len(submission_template)}개 쌍")
else:
    submission_template = None
    print("[WARN] sample_submission.csv를 찾을 수 없습니다. 후보쌍만 사용합니다.")

# 4. 2025-08 예측 (pair 단위) - submission template의 모든 쌍에 대해 예측!
pair_pred_df = predict_aug_2025_values_v2(
    rf_model,
    feat_df,
    candidates_df,
    feature_cols,
    submission_template=submission_template,  # 모든 쌍에 대해 예측 시도
)

# 5. sample_submission을 기준으로 제출 파일 생성 (fallback 포함)
submission_df = make_submission_from_pair_pred(
    pair_pred_df,
    submission_template=submission_template,
    feat_df=feat_df,  # fallback을 위한 피처 데이터 전달
)

# 후처리: 이상치 처리 및 스무딩
print("\n🔧 후처리 시작...")

# 1. 음수 값 제거
submission_df["value"] = submission_df["value"].clip(lower=0)

# 2. 이상치 처리 (99.9 percentile 이상은 제한)
if len(submission_df) > 0:
    upper_bound = submission_df["value"].quantile(0.999)
    submission_df["value"] = submission_df["value"].clip(upper=upper_bound)
    print(f"   이상치 처리: 상한 {upper_bound:.2f}")

# 3. 0 값이 너무 많으면 최소값으로 대체 (데이터 품질 개선)
zero_count = (submission_df["value"] == 0).sum()
if zero_count > len(submission_df) * 0.5:  # 50% 이상이 0이면
    non_zero_mean = submission_df[submission_df["value"] > 0]["value"].mean()
    if non_zero_mean > 0:
        submission_df.loc[submission_df["value"] == 0, "value"] = non_zero_mean * 0.1
        print(f"   ⚠️ {zero_count}개 0 값을 최소값({non_zero_mean * 0.1:.2f})으로 대체")

# 4. 최종 검증
submission_df["value"] = submission_df["value"].round().astype(int)
submission_df["value"] = submission_df["value"].clip(lower=0)

# 5. submission 저장
submission_df.to_csv("submission_kmu_v1.csv", index=False)

print("\n" + "="*60)
print("✅ pair-level 예측으로 submission_kmu_v1.csv 생성 완료!")
print(f"제출 row 수: {len(submission_df)}")
print(f"예측된 쌍 수: {len(pair_pred_df)}")
print(f"평균 예측값: {submission_df['value'].mean():.2f}")
print(f"중앙값 예측값: {submission_df['value'].median():.2f}")
print(f"예측값 범위: {submission_df['value'].min():.2f} ~ {submission_df['value'].max():.2f}")
print(f"0 값 개수: {(submission_df['value'] == 0).sum()} ({100*(submission_df['value'] == 0).sum()/len(submission_df):.1f}%)")
print("="*60)
print("\n처음 10개 행:")
print(submission_df.head(10))
print("\n📊 예측값 분포:")
print(submission_df['value'].describe())


In [ ]:
# ============================================================
# 🚀 최종 최적화 버전 - 모든 개선 사항 적용
# ============================================================
# 이 셀은 모든 최적화를 포함한 최종 버전입니다.
# 제출 전 반드시 이 셀을 실행하세요!

import os
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings('ignore')

# 필요한 변수 확인
required_vars = ['df_panel', 'candidates_df']
missing_vars = [var for var in required_vars if var not in globals()]

if missing_vars:
    raise NameError(
        f"다음 변수들이 정의되지 않았습니다: {', '.join(missing_vars)}\n"
        f"다음 셀들을 먼저 실행해주세요:\n"
        f"- df_panel: 패널 데이터 생성 셀\n"
        f"- candidates_df: 공행성 후보쌍 생성 셀"
    )

print("="*70)
print("🚀 최종 최적화 submission 생성 시작")
print("="*70)

# 1. 강화된 피처 생성
print("\n📊 1단계: 피처 생성...")
feat_base = make_item_features(df_panel, use_hs_stats=True, use_ema=True, use_zero_run=True)
feat_df = add_ts_core_features(feat_base)
print(f"   ✅ 피처 생성 완료: {len(feat_df.columns)}개 컬럼")

# 2. 최적화된 모델 학습
print("\n🔧 2단계: 모델 학습...")
train_last_t = pd.Timestamp("2024-12-01")
valid_last_t = pd.Timestamp("2025-06-01")

rf_model, feature_cols = train_pair_model_v2(
    feat_df,
    candidates_df,
    train_last_t=train_last_t,
    valid_last_t=valid_last_t,
    n_estimators=300,  # 최대 성능
    max_depth=20,      # 깊은 트리
    random_state=42,
)
print(f"   ✅ 모델 학습 완료: {len(feature_cols)}개 피처 사용")

# 3. Submission template 로드
print("\n📋 3단계: Submission template 로드...")
sample_path = os.path.join("..", "sample_submission.csv")
if os.path.exists(sample_path):
    submission_template = pd.read_csv(sample_path)
    print(f"   ✅ Template 로드 완료: {len(submission_template)}개 쌍")
else:
    submission_template = None
    print("   ⚠️ Template 없음, candidates_df 사용")

# 4. 모든 쌍에 대해 예측
print("\n🔮 4단계: 예측 수행...")
pair_pred_df = predict_aug_2025_values_v2(
    rf_model,
    feat_df,
    candidates_df,
    feature_cols,
    submission_template=submission_template,
)

# 5. Submission 생성 (fallback 포함)
print("\n📝 5단계: Submission 생성...")
submission_df = make_submission_from_pair_pred(
    pair_pred_df,
    submission_template=submission_template,
    feat_df=feat_df,
)

# 6. 후처리 및 검증
print("\n✨ 6단계: 후처리 및 검증...")

# submission_df가 비어있는지 확인
if len(submission_df) == 0:
    raise ValueError("❌ submission_df가 비어있습니다! 예측이 실패했습니다.")

# 이상치 처리
submission_df["value"] = submission_df["value"].clip(lower=0)
if len(submission_df) > 0:
    try:
        upper_bound = submission_df["value"].quantile(0.999)
        if not pd.isna(upper_bound):
            submission_df["value"] = submission_df["value"].clip(upper=upper_bound)
            print(f"   ✅ 이상치 처리: 상한 {upper_bound:,.2f}")
    except Exception as e:
        print(f"   ⚠️ 이상치 처리 중 오류 (무시): {e}")

# 0 값 처리 (너무 많으면 최소값으로)
zero_count = (submission_df["value"] == 0).sum()
if zero_count > len(submission_df) * 0.3:  # 30% 이상이 0이면
    non_zero_values = submission_df[submission_df["value"] > 0]["value"]
    if len(non_zero_values) > 0:
        try:
            min_val = max(1, int(non_zero_values.quantile(0.1)))
            submission_df.loc[submission_df["value"] == 0, "value"] = min_val
            print(f"   ⚠️ {zero_count}개 0 값을 최소값({min_val})으로 대체")
        except Exception as e:
            # quantile 실패 시 평균의 10% 사용
            min_val = max(1, int(non_zero_values.mean() * 0.1))
            submission_df.loc[submission_df["value"] == 0, "value"] = min_val
            print(f"   ⚠️ {zero_count}개 0 값을 최소값({min_val})으로 대체 (평균 기반)")

# 최종 정리
submission_df["value"] = pd.to_numeric(submission_df["value"], errors="coerce").fillna(0)
submission_df["value"] = submission_df["value"].round().astype(int)
submission_df["value"] = submission_df["value"].clip(lower=0)

# 7. 저장 및 리포트
submission_df.to_csv("submission_kmu_v1.csv", index=False)

print("\n" + "="*70)
print("✅ 최종 submission_kmu_v1.csv 생성 완료!")
print("="*70)
print(f"\n📊 통계:")
print(f"   - 제출 row 수: {len(submission_df):,}")
print(f"   - 예측된 쌍 수: {len(pair_pred_df):,}")
print(f"   - 평균값: {submission_df['value'].mean():,.2f}")
print(f"   - 중앙값: {submission_df['value'].median():,.2f}")
print(f"   - 최소값: {submission_df['value'].min():,}")
print(f"   - 최대값: {submission_df['value'].max():,}")
print(f"   - 0 값: {(submission_df['value'] == 0).sum():,} ({100*(submission_df['value'] == 0).sum()/len(submission_df):.1f}%)")
print(f"\n📈 분위수:")
try:
    print(f"   - 25%: {submission_df['value'].quantile(0.25):,.0f}")
    print(f"   - 50%: {submission_df['value'].quantile(0.50):,.0f}")
    print(f"   - 75%: {submission_df['value'].quantile(0.75):,.0f}")
    print(f"   - 95%: {submission_df['value'].quantile(0.95):,.0f}")
    print(f"   - 99%: {submission_df['value'].quantile(0.99):,.0f}")
except Exception as e:
    print(f"   ⚠️ 분위수 계산 중 오류: {e}")

print(f"\n📋 처음 20개 행:")
try:
    print(submission_df.head(20).to_string(index=False))
except Exception as e:
    print(submission_df.head(20))

print("\n" + "="*70)
print("🎯 제출 준비 완료! submission_kmu_v1.csv 파일을 확인하세요.")
print("="*70)


## 🎯 최종 점검 체크리스트

제출 전 반드시 확인하세요:

### ✅ 필수 확인 사항

1. **데이터 준비**
   - [ ] `df_panel` 생성 완료
   - [ ] `candidates_df` 생성 완료 (공행성 후보쌍)
   - [ ] `sample_submission.csv` 파일 존재 확인

2. **모델 설정**
   - [ ] `n_estimators=300` (충분한 트리 수)
   - [ ] `max_depth=20` (깊은 트리)
   - [ ] `submission_template` 전달 확인

3. **예측 범위**
   - [ ] submission template의 **모든 쌍**에 대해 예측 시도
   - [ ] 예측 불가능한 쌍은 fallback 전략 적용

4. **후처리**
   - [ ] 음수 값 제거
   - [ ] 이상치 처리 (99.9 percentile)
   - [ ] 0 값이 너무 많으면 최소값으로 대체

5. **최종 검증**
   - [ ] 모든 값이 0이 아닌지 확인
   - [ ] submission 파일 row 수 확인 (9900개 이상)
   - [ ] 값의 분포가 합리적인지 확인

### 🚀 성능 향상 팁

- **피처 엔지니어링**: weight, quantity도 활용
- **모델 강화**: 더 많은 트리, 더 깊은 트리
- **후처리**: 이상치 제거, 스무딩
- **Fallback 전략**: 예측 불가능한 쌍에 대한 스마트한 대체값

### ⚠️ 주의사항

- submission 파일이 모두 0이면 안 됩니다!
- 예측값이 없을 때는 아이템별 평균 사용
- 이상치가 너무 크면 제한 필요


## 🚀 공행성 점수 개선 버전 (v2)

기존 버전의 개선 사항:
1. **다양한 lag 길이 고려** (1, 2, 3개월 lead)
2. **Spearman 상관계수 추가** (비선형 관계 탐지)
3. **Cross-correlation 최대값 사용** (최적 lag 자동 탐지)
4. **임계값 조정** (더 많은 후보쌍 발견)
5. **상호정보량 기반 점수** (선형/비선형 모두 포착)
6. **정규화된 상관계수** (시계열 길이 보정)


In [ ]:
# ============================================================
# 🚀 개선된 공행성 후보쌍 생성 함수 (v2)
# ============================================================
# 주요 개선: 다양한 lag, Spearman 상관, cross-correlation 최대값

import numpy as np
import pandas as pd
from scipy.stats import spearmanr
from typing import Optional

def compute_correlations_v2(resid_mat: pd.DataFrame, max_lag: int = 3):
    """
    개선된 상관계수 계산:
    - 다양한 lag 길이 (1, 2, 3개월)
    - Pearson + Spearman 상관계수
    - Cross-correlation 최대값
    """
    if resid_mat.empty or len(resid_mat.columns) == 0:
        raise ValueError("resid_mat가 비어있거나 컬럼이 없습니다.")
    
    if len(resid_mat) < max_lag + 1:
        max_lag = max(1, len(resid_mat) - 1)
        print(f"⚠️ 데이터가 짧아 max_lag를 {max_lag}로 조정했습니다.")
    
    resid_centered = resid_mat - resid_mat.mean(axis=0)
    items = resid_centered.columns
    
    if len(items) == 0:
        raise ValueError("유효한 아이템이 없습니다.")
    
    # 동시 상관 (Pearson + Spearman)
    corr_same_pearson = resid_centered.corr()
    corr_same_spearman = resid_centered.corr(method='spearman')
    
    # Lead 상관을 여러 lag에 대해 계산
    corr_lead_dict = {}  # {lag: DataFrame}
    
    for lag in range(1, max_lag + 1):
        corr_lead = pd.DataFrame(index=items, columns=items, dtype=float)
        
        resid_t = resid_centered.iloc[:-lag, :] if lag < len(resid_centered) else resid_centered.iloc[:0, :]
        resid_tlag = resid_centered.iloc[lag:, :] if lag < len(resid_centered) else resid_centered.iloc[:0, :]
        
        for i in items:
            x = resid_t[i].values if i in resid_t.columns else np.array([])
            mask_x = ~np.isnan(x)
            
            for j in items:
                if i == j:
                    corr_lead.loc[i, j] = np.nan
                    continue
                
                y = resid_tlag[j].values if j in resid_tlag.columns else np.array([])
                mask_y = ~np.isnan(y)
                
                mask = mask_x & mask_y
                if mask.sum() < 5:
                    corr_lead.loc[i, j] = np.nan
                    continue
                
                # Pearson 상관
                try:
                    corr_pearson = np.corrcoef(x[mask], y[mask])[0, 1]
                    if np.isnan(corr_pearson):
                        corr_lead.loc[i, j] = np.nan
                        continue
                except:
                    corr_lead.loc[i, j] = np.nan
                    continue
                
                # Spearman 상관
                try:
                    corr_spearman, _ = spearmanr(x[mask], y[mask])
                    if np.isnan(corr_spearman):
                        corr_spearman = corr_pearson
                except:
                    corr_spearman = corr_pearson
                
                # 두 상관계수의 평균 (더 robust)
                corr_lead.loc[i, j] = (corr_pearson + corr_spearman) / 2
        
        corr_lead_dict[lag] = corr_lead
    
    return corr_same_pearson, corr_same_spearman, corr_lead_dict


def select_candidate_pairs_v2(
    corr_same_pearson: pd.DataFrame,
    corr_same_spearman: pd.DataFrame,
    corr_lead_dict: dict,
    th_same: float = 0.3,  # 낮춤 (더 많은 후보)
    th_lead: float = 0.4,  # 낮춤
    min_diff: float = 0.03,  # 낮춤
    use_best_lag: bool = True,  # 최적 lag 자동 선택
    top_k: Optional[int] = None
) -> pd.DataFrame:
    """
    개선된 후보쌍 선정:
    - Pearson과 Spearman 상관계수 모두 고려
    - 여러 lag 중 최적 lag 선택
    - 더 낮은 임계값으로 더 많은 후보 발견
    """
    items = corr_same_pearson.index
    pairs = []
    
    # 동시 상관은 Pearson과 Spearman의 평균 사용
    corr_same = (corr_same_pearson + corr_same_spearman) / 2
    
    for i in items:
        for j in items:
            if i == j:
                continue
            
            same = corr_same.loc[i, j]
            if pd.isna(same) or same < th_same:
                continue
            
            # 여러 lag 중 최적 lag 찾기
            best_lag = 1
            best_lead_ij = np.nan
            best_lead_ji = np.nan
            
            if use_best_lag:
                # 각 lag에 대해 상관계수 확인하고 최대값 선택
                for lag in sorted(corr_lead_dict.keys()):
                    lead_ij_lag = corr_lead_dict[lag].loc[i, j]
                    lead_ji_lag = corr_lead_dict[lag].loc[j, i]
                    
                    if pd.notna(lead_ij_lag) and (pd.isna(best_lead_ij) or lead_ij_lag > best_lead_ij):
                        best_lead_ij = lead_ij_lag
                        best_lag = lag
                    
                    if pd.notna(lead_ji_lag) and (pd.isna(best_lead_ji) or lead_ji_lag > best_lead_ji):
                        best_lead_ji = lead_ji_lag
                        if pd.isna(best_lead_ij) or lead_ji_lag > best_lead_ij:
                            best_lag = lag
            else:
                # lag=1만 사용
                best_lead_ij = corr_lead_dict[1].loc[i, j] if 1 in corr_lead_dict else np.nan
                best_lead_ji = corr_lead_dict[1].loc[j, i] if 1 in corr_lead_dict else np.nan
            
            # i→j 방향
            if pd.notna(best_lead_ij):
                if pd.isna(best_lead_ji) or (best_lead_ij - best_lead_ji) >= min_diff:
                    if best_lead_ij >= th_lead:
                        pairs.append({
                            "leading_item_id": i,
                            "following_item_id": j,
                            "corr_same": same,
                            "corr_lead": best_lead_ij,
                            "lag": best_lag,
                            "direction": "i_leads_j"
                        })
            
            # j→i 방향
            if pd.notna(best_lead_ji):
                if pd.isna(best_lead_ij) or (best_lead_ji - best_lead_ij) >= min_diff:
                    if best_lead_ji >= th_lead:
                        pairs.append({
                            "leading_item_id": j,
                            "following_item_id": i,
                            "corr_same": same,
                            "corr_lead": best_lead_ji,
                            "lag": best_lag,
                            "direction": "j_leads_i"
                        })
    
    candidates_df = pd.DataFrame(pairs)
    
    if candidates_df.empty:
        return candidates_df
    
    # 중복 제거 및 정렬
    candidates_df = candidates_df.sort_values(
        ["corr_lead", "corr_same"], ascending=False
    ).drop_duplicates(
        subset=["leading_item_id", "following_item_id"]
    )
    
    if top_k is not None:
        candidates_df = candidates_df.head(top_k)
    
    return candidates_df


def generate_comovement_candidates_v2(
    df_panel: pd.DataFrame,
    min_nonzero_months: int = 6,  # 낮춤 (더 많은 아이템 포함)
    min_std: float = 0.15,  # 낮춤
    th_same: float = 0.3,  # 낮춤
    th_lead: float = 0.4,  # 낮춤
    min_diff: float = 0.03,  # 낮춤
    max_lag: int = 3,  # 여러 lag 고려
    use_best_lag: bool = True,
    top_k: Optional[int] = None
) -> pd.DataFrame:
    """
    개선된 공행성 후보쌍 생성 파이프라인 (v2)
    기존 함수들을 직접 호출 (노트북에 이미 정의되어 있음)
    """
    # 입력 검증
    if df_panel is None or df_panel.empty:
        raise ValueError("df_panel이 비어있습니다.")
    
    required_cols = ["date", "item_id", "value"]
    missing_cols = [c for c in required_cols if c not in df_panel.columns]
    if missing_cols:
        raise ValueError(f"df_panel에 필요한 컬럼이 없습니다: {missing_cols}")
    
    # 0) value matrix (기존 함수 사용)
    df = df_panel.copy()
    df = df.sort_values(["date", "item_id"])
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    df["value"] = df["value"].clip(lower=0)
    df["log_value"] = np.log1p(df["value"].astype(float))
    df.loc[~np.isfinite(df["log_value"]), "log_value"] = np.nan
    
    # pivot 전에 데이터 확인
    if df["item_id"].nunique() == 0 or df["date"].nunique() == 0:
        raise ValueError("유효한 item_id나 date가 없습니다.")
    
    try:
        value_mat = df.pivot(index="date", columns="item_id", values="log_value").sort_index()
    except Exception as e:
        raise ValueError(f"pivot 실패: {e}. 중복된 (date, item_id) 조합이 있는지 확인하세요.")
    
    if value_mat.empty:
        raise ValueError("value_mat가 비어있습니다.")
    
    # 1) item filtering (더 관대한 기준)
    value_nonzero_cnt = (value_mat > 0).sum(axis=0)
    value_std = value_mat.std(axis=0, skipna=True)
    valid_items = value_mat.columns[
        (value_nonzero_cnt >= min_nonzero_months) & (value_std >= min_std)
    ]
    value_mat_f = value_mat[valid_items]
    
    # 2) residuals (기존 함수 사용)
    market_index = value_mat_f.mean(axis=1, skipna=True)
    common_index = value_mat_f.index.intersection(market_index.index)
    X = market_index.loc[common_index].values
    Y = value_mat_f.loc[common_index]
    resid_mat = pd.DataFrame(index=common_index, columns=Y.columns, dtype=float)
    for col in Y.columns:
        y = Y[col].values
        mask = ~np.isnan(y) & ~np.isnan(X)
        if mask.sum() < 5:
            resid_mat[col] = np.nan
            continue
        X_m = X[mask]
        y_m = y[mask]
        A = np.vstack([np.ones_like(X_m), X_m]).T
        coef, _, _, _ = np.linalg.lstsq(A, y_m, rcond=None)
        a, b = coef
        y_hat = a + b * X
        resid = y - y_hat
        resid_mat[col] = resid
    
    # 3) 개선된 상관계수 계산
    corr_same_pearson, corr_same_spearman, corr_lead_dict = compute_correlations_v2(
        resid_mat, max_lag=max_lag
    )
    
    # 4) 개선된 후보쌍 선정
    candidates_df = select_candidate_pairs_v2(
        corr_same_pearson, corr_same_spearman, corr_lead_dict,
        th_same=th_same,
        th_lead=th_lead,
        min_diff=min_diff,
        use_best_lag=use_best_lag,
        top_k=top_k
    )
    
    return candidates_df


### 개선된 공행성 후보쌍 생성 실행


In [ ]:
# 개선된 공행성 후보쌍 생성 (더 많은 후보 발견)
print("🚀 개선된 공행성 후보쌍 생성 중...")
print("   - 더 낮은 임계값 사용")
print("   - 여러 lag 길이 고려 (1, 2, 3개월)")
print("   - Pearson + Spearman 상관계수 조합")

# df_panel 변수 확인
if 'df_panel' not in globals() or df_panel is None:
    raise ValueError("❌ df_panel 변수가 정의되지 않았습니다. 먼저 패널 데이터를 생성해주세요.")

try:
    candidates_df_v2 = generate_comovement_candidates_v2(
        df_panel,
        min_nonzero_months=6,  # 기존 8 → 6 (더 많은 아이템)
        min_std=0.15,  # 기존 0.2 → 0.15
        th_same=0.3,  # 기존 0.4 → 0.3
        th_lead=0.4,  # 기존 0.5 → 0.4
        min_diff=0.03,  # 기존 0.05 → 0.03
        max_lag=3,  # 1, 2, 3개월 모두 고려
        use_best_lag=True,  # 최적 lag 자동 선택
        top_k=1000  # 상위 1000쌍
    )
    
    print(f"\n✅ 후보쌍 생성 완료!")
    print(f"   - 기존 버전: {len(candidates_df) if 'candidates_df' in globals() else 0}개 쌍")
    print(f"   - 개선 버전: {len(candidates_df_v2)}개 쌍")
    
    if len(candidates_df_v2) > 0:
        print(f"\n📊 개선 버전 상위 20개 쌍:")
        print(candidates_df_v2.head(20))
        
        # candidates_df를 v2로 업데이트 (이후 코드에서 사용)
        candidates_df = candidates_df_v2.copy()
        print(f"\n✅ candidates_df를 v2로 업데이트 완료!")
    else:
        print("\n⚠️ 후보쌍이 발견되지 않았습니다. 임계값을 더 낮춰보세요.")
        # 기존 candidates_df 유지
        if 'candidates_df' not in globals():
            candidates_df = pd.DataFrame(columns=['leading_item_id', 'following_item_id', 'corr_same', 'corr_lead'])
            
except Exception as e:
    print(f"\n❌ 오류 발생: {e}")
    print(f"   오류 타입: {type(e).__name__}")
    import traceback
    print(f"\n상세 오류:")
    traceback.print_exc()
    raise
